**Τι κάνει αυτό το notebook;**

Τα datasets από την innews βασίζονται στα social. Αυτό σημαίνει ότι:
- κάθε row περιέχει κάποιο post σε social, το οποίο αναφέρεται σε κάποιο ειδησεογραφικό άρθρο
- το άρθρο μπορεί να περιέχει το οτιδήποτε (π.χ. τη γελοιογραφία της ημέρας), δηλαδή δεν έχει πάντα κείμενο
- τα posts στα social μπορεί να είναι από οποιονδήποτε λογαριασμό

Εμείς θέλουμε το dataset να βασίζεται στα ειδησεογραφικά άρθρα. Αυτό σημαίνει ότι θέλουμε:
- κάθε row να περιέχει ένα ειδησεογραφικό άρθρο
- να μετριέται το engagement του σε Likes, Shares, Comments
- το πλήθος των fb posts και των tweets (από το ίδιο το μέσο και από άλλους χρήστες)
 
Άρα, αυτό το notebook:
- 

In [1]:
import pandas as pd

import numpy as np

from datetime import date 
from datetime import datetime, timedelta
import sys

import os
from pathlib import Path

import time

import re

import shutil

from os import listdir
from os.path import isfile, join

In [2]:
import sys 
#!{sys.executable} -m pip install --user matplotlib

# **Make comp dfs**

**Read sites list with ratings**

In [ ]:
df = pd.read_csv('iq_journalism/0_csvs_for_IQ_feed/0_for_chat_gpt/high_sites_top_75_text_chunks_300.csv
')


#    iq_journalism/0_csvs_for_IQ_feed/0_for_chat_gpt/high_sites_top_75_text_chunks_300.csv



In [3]:
sites_list_df = pd.read_csv('iq_sites_list_rated(updated).csv')
sites_list_df.head(1)

,news_site,url,rate,Unnamed: 3
0,creteplus.gr,https://creteplus.gr,high,NaN


**Define column names**

In [4]:
header_names = ["type_s", # τύπος social
                "url_s", # url του social
                "external_link_s", # url του άρθρου
                "source_s", # όνομα του social
                "timestamp_dt", # ημερομηνία δημοσίευσης στο social
                "text_txt_el", # κείμενο στο social
                "retweet_count_l", # πλήθος retweets (twitter)
                "followers_i", # πλήθος followers (twitter)
                "following_i", # πλήθος following (twitter)
                "mentions_i", # πλήθος αναφορών (twitter)
                "comments_i", # πλήθος σχολίων (fb)
                "likes_i", # πλήθος likes (fb
                "full_artcl_text" # κείμενο άρθρου στο site (δεν περιλαμβάνεται ο τίτλος)
               ]

**Make comp dfs for high and low quality sites**

In [5]:
# create dictionaries for the high and low sites dfs
dfs_high = {}
dfs_low = {}

# path to csvs from innews
main_csvs_path = '0_innews_data_sets/iqcsv/'

# read csv, make df, put it in the dictionary
for index, row in sites_list_df.iterrows():
    key = row['news_site']
    csv_name = key + '.csv'
    site_df = pd.read_csv(main_csvs_path + csv_name, header=None, names=header_names, on_bad_lines='skip')
    site_df['site'] = key
    if row['rate'] == 'high':
        dfs_high[key] = site_df
    elif row['rate'] == 'low':
        dfs_low[key] = site_df

In [6]:
# make comp dfs for high and low dfs
for q in ['high', 'low']:
    if q == 'high':
        for i in range(0, len(dfs_high)-1):
            if i == 0:
                high_comp_df = pd.concat([dfs_high[list(dfs_high.keys())[i]], dfs_high[list(dfs_high.keys())[i+1]]])
            else:
                high_comp_df = pd.concat([high_comp_df, dfs_high[list(dfs_high.keys())[i+1]]])
    elif q == 'low':
        for i in range(0, len(dfs_low)-1):
            if i==0:
                low_comp_df = pd.concat([dfs_low[list(dfs_low.keys())[i]], dfs_low[list(dfs_low.keys())[i+1]]])
            else:
                low_comp_df = pd.concat([low_comp_df, dfs_low[list(dfs_low.keys())[i+1]]])
high_comp_df.reset_index(drop=True, inplace=True)
low_comp_df.reset_index(drop=True, inplace=True)

**Add an ID column**

In [7]:
high_comp_df.insert(0, 'in_id', high_comp_df.index)
low_comp_df.insert(0, 'in_id', low_comp_df.index)

In [8]:
high_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
0,0,facebook,https://www.facebook.com/284019554966088/posts...,http://www.creteplus.gr/news/proponisi-kai-sim...,creteplus.gr,2015-12-31T10:50:53Z,Προπόνηση και σήμερα για τους παίκτες του ΟΦΗ ...,0.0,31766.0,0.0,0.0,0.0,2.0,NaN,creteplus.gr


In [9]:
low_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,0.0,0.0,0.0,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,toxwni.gr


# **Describe and pre-process datasets**

## **High quality**

In [10]:
high_comp_df.dtypes

in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l     object
followers_i         object
following_i         object
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object

**Notes:** retweet_count_l, followers_i and following_i should be floats

In [11]:
#high_comp_df['retweet_count_l'].astype(float)

In [12]:
#replace whitespace values with nan
high_comp_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [13]:
# unique values
print(high_comp_df.nunique())

in_id              3016192
type_s                  31
url_s              3016090
external_link_s    1435420
source_s             88441
timestamp_dt       2644305
text_txt_el        2435466
retweet_count_l       8286
followers_i          21945
following_i           9670
mentions_i            1971
comments_i             333
likes_i               2437
full_artcl_text     902048
site                   128
dtype: int64


**Notes:** 
- type_s: there are 31 unique values in type_s column, where there should be only facebook and twitter
- url_s: unique url_s value counts should be equal to unique in_id value counts
- external_link_s: unique external_link_s value counts should be equal to unique full_artcl_text value counts
- timestamp_dt: unique timestamp_dt value counts should be equal to unique url_s value counts
- text_txt_el: unique text_txt_el value counts should be equal to unique url_s value counts

**type_s**

In [14]:
# find rows with type_s values other than 'facebook' and 'twitter'
high_comp_df[(high_comp_df.type_s != 'facebook') & (high_comp_df.type_s != 'twitter')]

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
301235,301235,Η ενημέρωση αυτή υλοποιήθηκε μέσω 54 γραπτών α...,οι οποίες είχαν γίνει στην ΕΠΟ από την Eurost...,"όπου ρητά αναφέρετο ότι \""μόνον η Εθνική Fact...",έναντι της επιχείρησής σας,"εκχωρηθεισών απαιτήσεων\"".",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301236,301236,Οι απαιτήσεις αυτές προέκυψαν λόγω του γεγονό...,η προηγούμενη διοίκηση της ΕΠΟ αντί να καταβά...,ως όφειλε,"στην \""Εθνική Factors Α.Ε.\""",τις κατέβαλε στην Eurostar (Travel Plan),"με αποτέλεσμα να τις οφείλει στην \""Εθνική Fa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301237,301237,Στις 21.11.2016 η ΕΠΟ έλαβε προσωρινή διαταγή ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301238,301238,Ωστόσο χθες,29 Νοεμβρίου 2016,"η εταιρεία \""Εθνική Factors Α.Ε.\"" προχώρησε ...",με αποτέλεσμα να μην μπορεί να γίνει καμία απ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301239,301239,Η Προσωρινή Διοικούσα Επιτροπή ήδη παραπέμπει ...,και λαμβάνει όλα τα κατάλληλα μέτρα για την α...,την υπεράσπιση των συμφερόντων της ΕΠΟ και τη...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301240,301240,Επιπροσθέτως,ενημέρωσε σχετικά τον επιτετραμμένο της FIFA ...,ο οποίος εξέφρασε την απόλυτη στήριξή του σε ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
301241,301241,Η ΠΔΕ θα υλοποιήσει στο έπακρο την εκπεφρασμέν...,και θα εκπληρώσει απόλυτα την αποστολή της γ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,stokokkino.gr
337778,337778,Μιλάμε για μόνιμο έγκλημα διαρκείας. Τα δάση χ...,αν όμως χτιστεί,ΤΕΛΟΣ!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,newsbeast.gr
337779,337779,Μέσα σε 30-40 χρόνια ΜΟΝΟ,έχουν αλλοιωθεί τεράστιες εκτάσεις,δάση,νησιά,παράλιες. Και λέω \'μόνο\' γιατί το δάσος υπήρχε,όχι πριν 40 χρόνια,αλλά για ΕΚΑΤΟΜΜΥΡΙΑ ΧΡΟΝΙΑ ΠΡΙΝ! Άρα,μιλάμε για ταχύτατη και ανεπίστροφη καταστροφ...,NaN,NaN,NaN,NaN,NaN,newsbeast.gr
337780,337780,"\""Βαριά βιομηχανία μας ο τουρισμός\"". Πως αμέ!...",να δεις μετά τουρισμό! Θυμάμαι η παρθένα ακόμ...,να είναι γεμάτη από Σουηδούς,Ολλανδούς,Γερμανούς,Φινλανδούς,κλπ κλπ. Με το που την καταντήσανε κιτσαριό (ναι,"οι ίδιοι κάτοικοι που \""κόπτονται\"" το περιβά...",εξαφανίστηκαν οι παραπάνω!,NaN,NaN,NaN,NaN,newsbeast.gr


In [15]:
# keep rows with type_s values 'facebook' or 'twitter' only
high_comp_df = high_comp_df[(high_comp_df.type_s == 'facebook') | (high_comp_df.type_s == 'twitter')]

In [16]:
# unique values
print(high_comp_df.nunique())

in_id              3016140
type_s                   2
url_s              3016075
external_link_s    1435410
source_s             88434
timestamp_dt       2644300
text_txt_el        2435462
retweet_count_l       8284
followers_i          21943
following_i           9669
mentions_i            1971
comments_i             333
likes_i               2437
full_artcl_text     902048
site                   128
dtype: int64


**Note:** differences between in_id, url_s, timestamp_dt, text_txt_el unique value counts and differences between external_link_s and full_artcl_text maybe exist because of nan values

**NaN values**

In [17]:
print(high_comp_df.isna().sum())

in_id                   0
type_s                  0
url_s                   0
external_link_s         0
source_s                0
timestamp_dt            0
text_txt_el           788
retweet_count_l         0
followers_i             0
following_i             0
mentions_i              0
comments_i              0
likes_i                 0
full_artcl_text    817783
site                    0
dtype: int64


In [18]:
# remove rows with nan values in any column
high_comp_df = high_comp_df.dropna(how='any')

In [19]:
print(high_comp_df.isna().sum())

in_id              0
type_s             0
url_s              0
external_link_s    0
source_s           0
timestamp_dt       0
text_txt_el        0
retweet_count_l    0
followers_i        0
following_i        0
mentions_i         0
comments_i         0
likes_i            0
full_artcl_text    0
site               0
dtype: int64


In [20]:
print(high_comp_df.nunique())

in_id              2197866
type_s                   2
url_s              2197824
external_link_s     912153
source_s             70689
timestamp_dt       1939166
text_txt_el        1711253
retweet_count_l       6572
followers_i          20028
following_i           9412
mentions_i            1838
comments_i             302
likes_i               2113
full_artcl_text     901915
site                    68
dtype: int64


**Notes:**
- site: after removing rows with missing values, remained 68 sites (from 128 initialy)
- url_s: unique social posts urls are still less than unique posts (in_id); this means that there must be some duplicate post urls

**Examine duplicates in url_s column**

In [21]:
# find duplicates in url_s column
len(high_comp_df['url_s'])-len(high_comp_df['url_s'].drop_duplicates())

42

In [22]:
# ποια είναι τα duplicates;
high_comp_df[high_comp_df.url_s.duplicated()]

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
595359,595359,facebook,https://www.facebook.com/126465787506093/posts...,http://www.ert.gr/kotzias-prepi-na-dixoume-sev...,Apostoliss Arvanitis,2016-11-13T21:15:25Z,Κοτζιάς: Πρέπει να δείξουμε σεβασμό στο εκλογι...,0,0,0,0.0,0.0,4.0,Την ανάγκη επίδειξης σεβασμού στο εκλογικό απ...,ert.gr
667924,667924,facebook,https://www.facebook.com/77310553857/posts/101...,http://www.zougla.gr/technology/smart-home-gad...,Alexandros Ntrls,2016-11-14T03:40:55Z,Δώρο το Xbox One S με κάθε αγορά LG OLED 4K τη...,0,44292,0,0.0,0.0,1.0,Η LG Electronics (LG) προχωράει σε μία εντυ...,zougla.gr
667926,667926,facebook,https://www.facebook.com/77310553857/posts/101...,http://www.zougla.gr/market/article/oute-i-end...,Dimi Gian,2016-11-14T06:51:10Z,Ούτε οι ενδιάμεσες εκπτώσεις έσωσαν τον τζίρο ...,0,44292,0,0.0,0.0,1.0,Επιμέλεια: Άντζελα Πεΐτση \r\n \r\nΤην περ...,zougla.gr
667927,667927,facebook,https://www.facebook.com/77310553857/posts/101...,http://www.zougla.gr/greece/article/anepi8imit...,Sportdog.gr,2016-11-14T01:17:06Z,Ανεπιθύμητος ο Ομπάμα σύμφωνα με ψήφισμα του Δ...,0,44292,0,0.0,0.0,2.0,Ψήφισμα με το οποίο κηρύσσεται «ανεπιθύμητος ...,zougla.gr
1042537,1042537,facebook,https://www.facebook.com/169076869796271/posts...,http://www.parapolitika.gr/article/389369/i-ei...,Kostas Sotirakis,2016-11-13T21:58:14Z,Η εισήγηση Τόμσεν για χρέος και νέο... μνημόνι...,0,62553,0,0.0,0.0,0.0,Επιμένει το Ταμείο για κατάργηση όλων των φορ...,parapolitika.gr
1042539,1042539,facebook,https://www.facebook.com/169076869796271/posts...,http://www.parapolitika.gr/article/389338/o-ka...,Parmenion Tourmpanidis,2016-11-13T22:06:39Z,Ο Καλογρίτσας επενδύει στην «Documento» του Βα...,0,62553,0,0.0,0.0,2.0,Στην τελική ευθεία η έκδοση της κυριακάτικης ...,parapolitika.gr
1042724,1042724,facebook,https://www.facebook.com/122865334479214/posts...,http://www.parapolitika.gr/article/389507/thil...,AEL,2016-11-14T10:13:41Z,Θηλιά στην κυβέρνηση οι απαιτήσεις των δανειστ...,0,12957,0,0.0,0.0,0.0,Με άγριες διαθέσεις στα ραντεβού με την ελλην...,parapolitika.gr
1114377,1114377,facebook,https://www.facebook.com/7236107677/posts/1015...,http://www.xrimaonline.gr/457918/zwdia-h-panse...,Chrispy Motoryacht,2016-11-14T20:42:28Z,Η Πανσέληνος Νοεμβρίου στον Ταύρο ! Δείτε τι θ...,0,176221,0,0.0,0.0,0.0,Ζώδια: Η Πανσέληνος στον Ταύρο,xrimaonline.gr
1429721,1429721,facebook,https://www.facebook.com/139085780797/posts/10...,http://news247.gr/eidiseis/oikonomia/anaptyksh...,Giorgos Mamouzellos,2016-11-13T22:49:30Z,"Ανάπτυξη 1,6% του ΑΕΠ το τρίτο τρίμηνο Δεύτερο...",0,270451,0,0.0,1.0,1.0,Δεύτερο θετικό τρίμηνο για το ΑΕΠ διαπιστώνει...,news247.gr
1429726,1429726,facebook,https://www.facebook.com/139085780797/posts/10...,http://news247.gr/eidiseis/synentefxeis/ti-kan...,Panos Triantos,2016-11-13T23:26:47Z,Τι κάνατε κύριε υπουργέ για τα Εξάρχεια; Βασίλ...,0,0,0,0.0,0.0,1.0,Βασίλης Κικίλιας και Νίκος Δένδιας μιλούν στο...,news247.gr


In [23]:
print(high_comp_df.loc[667924]['url_s'])
print(high_comp_df.loc[667926]['url_s'])
print(high_comp_df.loc[667927]['url_s'])

https://www.facebook.com/77310553857/posts/10155432554303858
https://www.facebook.com/77310553857/posts/10155432554303858
https://www.facebook.com/77310553857/posts/10155432554303858


**Note:** identical post urls have **wrong** values in other columns

**Drop rows with duplicate values in url_s column**

In [24]:
high_comp_df.drop_duplicates('url_s', keep=False, inplace=True)

In [25]:
print(high_comp_df.nunique())

in_id              2197796
type_s                   2
url_s              2197796
external_link_s     912136
source_s             70633
timestamp_dt       1939101
text_txt_el        1711212
retweet_count_l       6572
followers_i          20027
following_i           9412
mentions_i            1838
comments_i             302
likes_i               2113
full_artcl_text     901900
site                    68
dtype: int64


**Note:** full_artcl_text unique values are less than external_link_s unique values; this means that there must be duplicates in full_artcl_text that correspond to different external_link_s urls (different article urls share the same article text)

**Examine duplicates in full_artcl_text column**

In [26]:
# ποια είναι τα full_artcl_text με τις μεγαλύτερες επαναλήψεις;
high_comp_df.groupby('full_artcl_text').size().sort_values().tail(3)

full_artcl_text
Tο σχόλιο της ημέρας με το πενάκι του Γιάννη Δερμεντζόγλου.     409
 Δείτε εδώ αναλυτικά για όλα τα ζώδια:                          466
Το σχόλιο της ημέρας με το πενάκι του Γιάννη Δερμεντζόγλου.    3299
dtype: int64

**Note:** άρα υπάρχουν δικαιολογημένα duplicates στα κείμενα των άρθρων

In [27]:
# ποια είναι τα text_txt_el με τις μεγαλύτερες επαναλήψεις;
high_comp_df.groupby('text_txt_el').size().sort_values().tail(3)

text_txt_el
Ερωτικές προβλέψεις Πέμπτης για όλα τα ζώδια !    301
Ερωτοσκόπιο Σαββάτου για όλα τα ζώδια !           332
Ερωτοσκόπιο Δευτέρας για όλα τα ζώδια !           388
dtype: int64

**Note:** άρα υπάρχουν δικαιολογημένα duplicates και στα κείμενα των posts

In [28]:
# ποια είναι τα timestamp_dt με τις μεγαλύτερες επαναλήψεις;
high_comp_df.groupby('timestamp_dt').size().sort_values().tail(3)

timestamp_dt
2016-05-19T04:05:09Z    83
2016-12-17T23:05:08Z    89
2016-09-22T11:10:06Z    98
dtype: int64

**Note:** επίσης, θα μπορούσε να είναι δικαιολογημένο το να υπάρχουν posts με ίδιο timestamp

#### **Report on pre-processed high quality dataset**

In [29]:
print('High quality sites')
print('-' * len('High quality sites'))
print('\r')

# unique values
print('unique values:')
print(high_comp_df.nunique())
print('\r')

# data types
print('data types:')
print(high_comp_df.dtypes)
print('\r')

High quality sites
------------------

unique values:
in_id              2197796
type_s                   2
url_s              2197796
external_link_s     912136
source_s             70633
timestamp_dt       1939101
text_txt_el        1711212
retweet_count_l       6572
followers_i          20027
following_i           9412
mentions_i            1838
comments_i             302
likes_i               2113
full_artcl_text     901900
site                    68
dtype: int64

data types:
in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l     object
followers_i         object
following_i         object
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object



In [30]:
high_comp_df.dtypes

in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l     object
followers_i         object
following_i         object
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object

**Convert retweet_count_l, followers_i and following_i to float**

In [31]:
high_comp_df.retweet_count_l = high_comp_df.retweet_count_l.astype(float)
high_comp_df.followers_i = high_comp_df.followers_i.astype(float)
high_comp_df.following_i = high_comp_df.following_i.astype(float)
high_comp_df.dtypes

in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l    float64
followers_i        float64
following_i        float64
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object

### **Indentify site name in source_s (ποια posts έχουν γίνει από το ίδιο το μέσο που δημοσίευσε το άρθρο)**

In [32]:
# facebook and twitter counts
high_comp_df.groupby('type_s').in_id.count()

type_s
facebook     729624
twitter     1468172
Name: in_id, dtype: int64

In [33]:
pd.set_option('display.max_colwidth', None)

In [34]:
# Ποια source_s ονόματα σχετίζονται με κάθε site;
print(high_comp_df.groupby('site').source_s.unique())

site
24sports.com.cy                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

**Note:** υπάρχουν περιπτώσεις που το ίδιο το μέσο δεν έχει έχει ποστάρει το άρθρο του στα social από δικό του λογαριασμό και περιπτώσεις όπου ναι μεν το μέσο έχει δικό του λογαριασμό αλλά το source_s όνομά του δεν ταυτίζεται ακριβώς με το όνομα του site. Στη δέυτερη περίπτωση, μερικές φορές το source_s περιέχει (και) κεφαλαία γράμματα, ή δεν περιλαμβάνει την κατάληξη του url του (π.χ. το .gr)

In [35]:
# remove substring after first dot in site names
high_comp_df['sitename'] = high_comp_df['site'].replace(r'\..*', '', regex=True)

In [36]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [37]:
high_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,0.0,0.0,0.0,Ενα απίστευτο περιστατικό συνέβη σε συγκέντρω...,iefimerida.gr,iefimerida


In [38]:
# lower upper case letters in source_s names
high_comp_df['source_s_lower'] = high_comp_df['source_s'].str.lower()

In [39]:
high_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,0.0,0.0,0.0,Ενα απίστευτο περιστατικό συνέβη σε συγκέντρω...,iefimerida.gr,iefimerida,costas_gogos


In [40]:
# find source_s names that start with sitename
print(high_comp_df.sitename.nunique())
print(high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].sitename.nunique())
high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].head(1)

68
57


,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower
17559,17559,twitter,https://twitter.com/iefimerida/status/69053653...,http://www.iefimerida.gr/news/416619/o-emporik...,iefimerida,2016-01-22T16:08:34Z,Ερώτηση από το Ποτάμι για τον Σπύρο Καπράλο -Α...,0.0,170136.0,4250.0,25854.0,0.0,0.0,Από το ραδιόφωνο «Στο Κόκκινο» στην ΕΡΤ πέρασ...,iefimerida.gr,iefimerida,iefimerida


**Notes:** 57 out of 68 sites were identified. But are all matches correct?

In [41]:
pd.set_option('display.max_colwidth', None)
#pd.reset_option('display')

In [42]:
# matches
high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].groupby('sitename').source_s_lower.unique()

sitename
24sports                                                                                                         [24sports, 24sports.com.cy]
aek365                                                                                                                 [aek365gr, aek365.gr]
aitoloakarnanianews                                                                                                    [aitoloakarnanianews]
amna                                                                                                               [amna_news, amna_newseng]
antenna                                                                                                                        [antennanews]
antinews                                                                                                           [antinews.gr, antinewsgr]
astratv                                                                                                                  [astratv, astratv1]
athe

**Notes:** most matches are correct. Exceptions are for: capital, left, liberal, star

In [43]:
false_matches_high = ['capitaloskylo', 'capitalcontrol2', 'left2kolou', 'leftfantasy', 
                      'lefterispatra', 'liberal2402', 'liberalove1', 'starlightpocket', 
                      'starcadell', 'starbuzzgr']

In [44]:
all_matches_high = high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].source_s_lower.unique()
print(list(all_matches_high))

['iefimerida', 'tvxs', 'tvxs - τηλεόραση χωρίς σύνορα - στέλιος κούλογλου', 'thetoc_eng', 'thetoc_gr', 'stokokkino1055', 'newsbeast', 'skaigr', 'skai.gr', 'tovimagr', 'thepressproject', 'lifo (official)', 'lifo', 'lifoland', 'astratv', 'astratv1', '24sports', '24sports.com.cy', 'ertefer', 'ertsocial', 'ertsports1', 'ertaegean', 'ertsports', 'ertopen_', 'ertproinizoni', 'ert3social', 'ertboxofficial', 'ertvolou', 'zougla_online', 'zougla.gr (official)', 'thepapergr', 'antennanews', 'enikonomia', 'athensvoice', 'press724a', 'press724gr', 'star.gr', 'staralithies', 'starchannelgr', 'starlightpocket', 'starcadell', 'starbuzzgr', 'parapolitika', 'parapolitika.gr', 'parapolitika901', 'antinews.gr', 'antinewsgr', 'enikos_gr', 'enikos.gr', 'freepengr', 'freepen.gr', 'news247.gr', 'news247gr', 'balla.com.cy', 'balla_com_cy', 'infokids', 'cyclades24.gr', 'cyclades24', 'liberalgr', 'liberal2402', 'liberalove1', 'liberal_de', 'avgionline', 'naftemporikigr', 'naftemporiki_en', 'fnews_gr', 'fnews.gr

In [45]:
true_matches_high = []
for site in all_matches_high:
    if site not in false_matches_high:
        true_matches_high.append(site)
print(true_matches_high)

['iefimerida', 'tvxs', 'tvxs - τηλεόραση χωρίς σύνορα - στέλιος κούλογλου', 'thetoc_eng', 'thetoc_gr', 'stokokkino1055', 'newsbeast', 'skaigr', 'skai.gr', 'tovimagr', 'thepressproject', 'lifo (official)', 'lifo', 'lifoland', 'astratv', 'astratv1', '24sports', '24sports.com.cy', 'ertefer', 'ertsocial', 'ertsports1', 'ertaegean', 'ertsports', 'ertopen_', 'ertproinizoni', 'ert3social', 'ertboxofficial', 'ertvolou', 'zougla_online', 'zougla.gr (official)', 'thepapergr', 'antennanews', 'enikonomia', 'athensvoice', 'press724a', 'press724gr', 'star.gr', 'staralithies', 'starchannelgr', 'parapolitika', 'parapolitika.gr', 'parapolitika901', 'antinews.gr', 'antinewsgr', 'enikos_gr', 'enikos.gr', 'freepengr', 'freepen.gr', 'news247.gr', 'news247gr', 'balla.com.cy', 'balla_com_cy', 'infokids', 'cyclades24.gr', 'cyclades24', 'liberalgr', 'liberal_de', 'avgionline', 'naftemporikigr', 'naftemporiki_en', 'fnews_gr', 'fnews.gr', 'dimokratikinews', 'xenesglosses', 'paraskhnio', 'ysterografa', 'efsyntakt

In [46]:
high_comp_df['source_s_new'] = high_comp_df.apply(lambda x: x['sitename'] if x['source_s_lower'] 
                                                  in true_matches_high else x['source_s_lower'], axis=1)

In [47]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [48]:
high_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower,source_s_new
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,0.0,0.0,0.0,Ενα απίστευτο περιστατικό συνέβη σε συγκέντρω...,iefimerida.gr,iefimerida,costas_gogos,costas_gogos


**Posts on social from websites accounts**

In [49]:
high_comp_df[high_comp_df.source_s_new == high_comp_df.sitename].groupby('type_s').in_id.count()

type_s
facebook    260861
twitter     508823
Name: in_id, dtype: int64

### **Calculations for total counts**

**Nbr of posts on social per article**

In [50]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [51]:
# count social posts per article
high_comp_df['social_posts'] = high_comp_df.groupby(['external_link_s', 'type_s']).in_id.transform('count')

In [52]:
# write facebook counts in separate column
high_comp_df['facebook_posts'] = high_comp_df.apply(lambda x: 
                                                    x['social_posts'] if x['type_s']=='facebook' else 0, axis=1)

In [53]:
# write tweets counts in separate column
high_comp_df['tweets'] = high_comp_df.apply(lambda x: 
                                                    x['social_posts'] if x['type_s']=='twitter' else 0, axis=1)

**Nbr of retweets per article**

In [54]:
high_comp_df['retweets_total'] = high_comp_df.groupby('external_link_s').retweet_count_l.transform('sum')
high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.retweets_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,likes_i,full_artcl_text,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,...,0.0,Ενα απίστευτο περιστατικό συνέβη σε συγκέντρω...,iefimerida.gr,iefimerida,costas_gogos,costas_gogos,4,0,4,19739.0


**Nbr of likes per article**

In [55]:
high_comp_df.columns

Index(['in_id', 'type_s', 'url_s', 'external_link_s', 'source_s',
       'timestamp_dt', 'text_txt_el', 'retweet_count_l', 'followers_i',
       'following_i', 'mentions_i', 'comments_i', 'likes_i', 'full_artcl_text',
       'site', 'sitename', 'source_s_lower', 'source_s_new', 'social_posts',
       'facebook_posts', 'tweets', 'retweets_total'],
      dtype='object')

In [56]:
high_comp_df['likes_total'] = high_comp_df.groupby('external_link_s').likes_i.transform('sum')
high_comp_df[(high_comp_df.type_s=='facebook') & (high_comp_df.likes_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,full_artcl_text,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total
20196,20196,facebook,https://www.facebook.com/187527241268787/posts...,http://www.iefimerida.gr/news/253364/apokleist...,Wired and Ready,2016-02-26T19:28:23Z,Αποκλειστικό: Η Ελλάδα έστειλε σχέδιο έκτακτης...,0.0,1199.0,0.0,...,Σύμφωνα με αποκλειστικές και απολύτως διασταυ...,iefimerida.gr,iefimerida,wired and ready,wired and ready,1,1,0,0.0,3.0


**Nbr of comments per article**

In [57]:
high_comp_df['comments_total'] = high_comp_df.groupby('external_link_s').comments_i.transform('sum')
high_comp_df[(high_comp_df.type_s=='facebook') & (high_comp_df.comments_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total
20196,20196,facebook,https://www.facebook.com/187527241268787/posts...,http://www.iefimerida.gr/news/253364/apokleist...,Wired and Ready,2016-02-26T19:28:23Z,Αποκλειστικό: Η Ελλάδα έστειλε σχέδιο έκτακτης...,0.0,1199.0,0.0,...,iefimerida.gr,iefimerida,wired and ready,wired and ready,1,1,0,0.0,3.0,1.0


**Nbr of followers per article**

In [58]:
high_comp_df['followers_total'] = high_comp_df.groupby('external_link_s').followers_i.transform('sum')
high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.followers_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,...,iefimerida,costas_gogos,costas_gogos,4,0,4,19739.0,0.0,0.0,2148.0


**Nbr of following per article**

In [59]:
high_comp_df['following_total'] = high_comp_df.groupby('external_link_s').following_i.transform('sum')
high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.following_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,...,costas_gogos,costas_gogos,4,0,4,19739.0,0.0,0.0,2148.0,1678.0


**Nbr of mentions per article**

In [60]:
high_comp_df['mentions_total'] = high_comp_df.groupby('external_link_s').mentions_i.transform('sum')
high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.mentions_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
16436,16436,twitter,https://twitter.com/panayioti1974/status/68268...,http://www.iefimerida.gr/news/410020/pio-sklir...,panayioti1974,2015-12-31T23:53:12Z,"Παιθκιά , καλλη χρονιά να εχουμε, σας ευχαριστ...",0.0,1057.0,477.0,...,panayioti1974,71,0,71,3673.0,0.0,0.0,468090.0,208283.0,29250.0


**Remove unneeded columns**

In [61]:
high_comp_df.columns

Index(['in_id', 'type_s', 'url_s', 'external_link_s', 'source_s',
       'timestamp_dt', 'text_txt_el', 'retweet_count_l', 'followers_i',
       'following_i', 'mentions_i', 'comments_i', 'likes_i', 'full_artcl_text',
       'site', 'sitename', 'source_s_lower', 'source_s_new', 'social_posts',
       'facebook_posts', 'tweets', 'retweets_total', 'likes_total',
       'comments_total', 'followers_total', 'following_total',
       'mentions_total'],
      dtype='object')

In [62]:
# source_s should be also excluded
unneeded_cols = ['site', 'source_s_lower', 'social_posts']

In [63]:
high_comp_df.drop(unneeded_cols, axis=1, inplace=True)
high_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
16430,16430,twitter,https://twitter.com/Costas_Gogos/status/682694...,http://www.iefimerida.gr/news/409912/diadiloti...,Costas_Gogos,2016-01-01T00:45:12Z,@mairiboo Καλή χρονιά! ♡♡♡♡,0.0,107.0,81.0,...,iefimerida,costas_gogos,0,4,19739.0,0.0,0.0,2148.0,1678.0,0.0


In [64]:
len(high_comp_df)

2197796

In [65]:
len(high_comp_df.external_link_s.unique())

912136

### **Make df with posts only from sites accounts**

In [66]:
high_comp_df[high_comp_df.source_s_new == high_comp_df.sitename]

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
16587,16587,twitter,https://twitter.com/kathimerini_gr/status/6837...,http://www.iefimerida.gr/news/411111/steit-nti...,kathimerini_gr,2016-01-03T19:30:11Z,#Kopinion Ας μιλήσουμε για τον καιρό https://t...,0.0,494912.0,432.0,...,iefimerida,iefimerida,0,9,163.0,0.0,0.0,532333.0,11693.0,7838.0
16879,16879,twitter,https://twitter.com/ertvolou/status/6858275752...,http://www.iefimerida.gr/news/412810/konstanti...,ertvolou,2016-01-09T16:16:50Z,Επιτακτική η ανάγκη αξιοποίησης\nτων 23 περιφε...,0.0,292.0,173.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0.0,292.0,173.0,0.0
17500,17500,twitter,https://twitter.com/ASTRATV1/status/6900888733...,http://www.iefimerida.gr/news/416004/epithesei...,ASTRATV1,2016-01-21T10:29:43Z,Διπλό «όχι» από Παναθηναϊκό σε Γιάννινα https:...,0.0,2272.0,1957.0,...,iefimerida,iefimerida,0,2,0.0,0.0,0.0,2655.0,2297.0,0.0
17559,17559,twitter,https://twitter.com/iefimerida/status/69053653...,http://www.iefimerida.gr/news/416619/o-emporik...,iefimerida,2016-01-22T16:08:34Z,Ερώτηση από το Ποτάμι για τον Σπύρο Καπράλο -Α...,0.0,170136.0,4250.0,...,iefimerida,iefimerida,0,15,23653.0,0.0,0.0,266651.0,46886.0,28311.0
17682,17682,twitter,https://twitter.com/TheTOC_gr/status/691218398...,http://www.iefimerida.gr/news/417265/agrinio-a...,TheTOC_gr,2016-01-24T13:18:02Z,"Τρόμος στην Αλάσκα από σεισμό 6,4 Ρίχτερ https...",0.0,66945.0,9125.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0.0,66945.0,9125.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3016182,3016182,twitter,https://twitter.com/PressPublica/status/815255...,http://www.presspublica.gr/%cf%84%cf%83%ce%af%...,PressPublica,2016-12-31T19:58:25Z,"Τσίπρας: Ετος ανάπτυξης, επενδύσεων και μεγάλω...",0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0
3016187,3016187,twitter,https://twitter.com/PressPublica/status/815267...,http://www.presspublica.gr/%cf%84%cf%83%ce%b1%...,PressPublica,2016-12-31T20:45:03Z,Τσαβούσογλου: Πενταμερής στη Γενεύη για το Κυπ...,0.0,1396.0,432.0,...,presspublica,presspublica,0,2,0.0,0.0,0.0,4014.0,2888.0,0.0
3016189,3016189,twitter,https://twitter.com/PressPublica/status/815274...,http://www.presspublica.gr/%ce%bc%ce%b7%cf%84%...,PressPublica,2016-12-31T21:11:56Z,Μητσοτάκης: Να αφήσουμε πίσω ξεπερασμένες ιδεο...,0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0
3016190,3016190,twitter,https://twitter.com/PressPublica/status/815288...,http://www.presspublica.gr/%ce%b4%ce%b9%ce%b1%...,PressPublica,2016-12-31T22:08:29Z,Διακοπές ρεύματος στα νοτια προάστια της Αθήνα...,0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0


In [67]:
high_comp_df.loc[16587, 'full_artcl_text']

' Οι Ηνωμένες Πολιτείες δεν αναγνωρίζουν την λεγόμενη Τουρκική Δημοκρατία της Βόρειας Κύπρου («ΤΔΒΚ»), όπως και καμιά άλλη χώρα πλην της Τουρκίας, αναφέρεται στην έκθεση ανθρωπίνων δικαιωμάτων του Στέιτ Ντιπάρτμεντ, που δόθηκε στη δημοσιότητα. Στην έκθεση, σύμφωνα με τον ανταποκριτή του ΚΥΠΕ στην Νέα Υόρκη Απόστολο Ζουπανιώτη, κάτω από τον τίτλο «Κύπρος» υπάρχουν με βάση την πάγια τακτική δύο χωριστά κεφάλαια, ένα για την Κυπριακή Δημοκρατία και ένα για την περιοχή που διοικείται από Τουρκοκύπριους. Η Κυπριακή Δημοκρατία παρουσιάζεται ως συνταγματική και πολυκομματική δημοκρατία, ενώ σημειώνεται ότι οι πολιτικές αρχές διατηρούν τον αποτελεσματικό έλεγχο των δυνάμεων ασφαλείας.  \nΤα πλέον σημαντικά ζητήματα ανθρωπίνων δικαιωμάτων συμπεριλαμβάνουν περιστατικά διαφθοράς, τα οποία, όπως επισημαίνεται, διερεύνησαν οι αρχές. Προστίθεται ακόμα ότι η κυβέρνηση έχει διώξει αξιωματούχους για παραβιάσεις ανθρωπίνων δικαιωμάτων και ότι δεν υπήρξαν αναφορές ατιμωρησίας κατά τη διάρκεια του έτους. 

In [68]:
print(high_comp_df.loc[16587, 'external_link_s'])

http://www.iefimerida.gr/news/411111/steit-ntipartment-oi-ipa-den-anagnorizoyn-tin-legomeni-toyrkiki-dimokratia-tis-voreias


In [69]:
high_comp_df.loc[16587, 'text_txt_el']

'#Kopinion Ας μιλήσουμε για τον καιρό https://t.co/HfFSRMbvIk'

**Note:** όπως φαίνεται από το πρώτο row, υπάρχουν περιπτώσεις που το κείμενο του post είναι άσχετο με το άρθρο στο οποίο υποτίθεται αντιστοιχεί

**Create df with posts from sites only**

In [70]:
high_comp_posts_from_sites_only_df = high_comp_df[high_comp_df.source_s_new == high_comp_df.sitename]

In [71]:
high_comp_posts_from_sites_only_df

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
16587,16587,twitter,https://twitter.com/kathimerini_gr/status/6837...,http://www.iefimerida.gr/news/411111/steit-nti...,kathimerini_gr,2016-01-03T19:30:11Z,#Kopinion Ας μιλήσουμε για τον καιρό https://t...,0.0,494912.0,432.0,...,iefimerida,iefimerida,0,9,163.0,0.0,0.0,532333.0,11693.0,7838.0
16879,16879,twitter,https://twitter.com/ertvolou/status/6858275752...,http://www.iefimerida.gr/news/412810/konstanti...,ertvolou,2016-01-09T16:16:50Z,Επιτακτική η ανάγκη αξιοποίησης\nτων 23 περιφε...,0.0,292.0,173.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0.0,292.0,173.0,0.0
17500,17500,twitter,https://twitter.com/ASTRATV1/status/6900888733...,http://www.iefimerida.gr/news/416004/epithesei...,ASTRATV1,2016-01-21T10:29:43Z,Διπλό «όχι» από Παναθηναϊκό σε Γιάννινα https:...,0.0,2272.0,1957.0,...,iefimerida,iefimerida,0,2,0.0,0.0,0.0,2655.0,2297.0,0.0
17559,17559,twitter,https://twitter.com/iefimerida/status/69053653...,http://www.iefimerida.gr/news/416619/o-emporik...,iefimerida,2016-01-22T16:08:34Z,Ερώτηση από το Ποτάμι για τον Σπύρο Καπράλο -Α...,0.0,170136.0,4250.0,...,iefimerida,iefimerida,0,15,23653.0,0.0,0.0,266651.0,46886.0,28311.0
17682,17682,twitter,https://twitter.com/TheTOC_gr/status/691218398...,http://www.iefimerida.gr/news/417265/agrinio-a...,TheTOC_gr,2016-01-24T13:18:02Z,"Τρόμος στην Αλάσκα από σεισμό 6,4 Ρίχτερ https...",0.0,66945.0,9125.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0.0,66945.0,9125.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3016182,3016182,twitter,https://twitter.com/PressPublica/status/815255...,http://www.presspublica.gr/%cf%84%cf%83%ce%af%...,PressPublica,2016-12-31T19:58:25Z,"Τσίπρας: Ετος ανάπτυξης, επενδύσεων και μεγάλω...",0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0
3016187,3016187,twitter,https://twitter.com/PressPublica/status/815267...,http://www.presspublica.gr/%cf%84%cf%83%ce%b1%...,PressPublica,2016-12-31T20:45:03Z,Τσαβούσογλου: Πενταμερής στη Γενεύη για το Κυπ...,0.0,1396.0,432.0,...,presspublica,presspublica,0,2,0.0,0.0,0.0,4014.0,2888.0,0.0
3016189,3016189,twitter,https://twitter.com/PressPublica/status/815274...,http://www.presspublica.gr/%ce%bc%ce%b7%cf%84%...,PressPublica,2016-12-31T21:11:56Z,Μητσοτάκης: Να αφήσουμε πίσω ξεπερασμένες ιδεο...,0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0
3016190,3016190,twitter,https://twitter.com/PressPublica/status/815288...,http://www.presspublica.gr/%ce%b4%ce%b9%ce%b1%...,PressPublica,2016-12-31T22:08:29Z,Διακοπές ρεύματος στα νοτια προάστια της Αθήνα...,0.0,1396.0,432.0,...,presspublica,presspublica,0,1,0.0,0.0,0.0,1396.0,432.0,0.0


**From df with posts from sites only, Create one df with fb posts and one for tweets**

In [72]:
high_sites_fb_df = high_comp_posts_from_sites_only_df[high_comp_posts_from_sites_only_df.type_s=='facebook']
print(len(high_sites_fb_df))
high_sites_tw_df = high_comp_posts_from_sites_only_df[high_comp_posts_from_sites_only_df.type_s=='twitter']
print(len(high_sites_tw_df))

260861
508823


In [73]:
high_sites_tw_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
16587,16587,twitter,https://twitter.com/kathimerini_gr/status/6837...,http://www.iefimerida.gr/news/411111/steit-nti...,kathimerini_gr,2016-01-03T19:30:11Z,#Kopinion Ας μιλήσουμε για τον καιρό https://t...,0.0,494912.0,432.0,...,iefimerida,iefimerida,0,9,163.0,0.0,0.0,532333.0,11693.0,7838.0


## **Experiments that leaded to conclusion that there are duplicate posts**

Με τα παρακάτω, αποκαλύφθηκε ότι υπάρχουν και πολλαπλά posts για κάποια άρθρα, τα οποία έχουν το ίδιο κείμενο στο post, παρότι το link των posts και η ώρα δημοσίευσής τους είναι διαφορετικά. Δοκίμασα κάποια links αυτών των posts και δεν είναι διαθέσιμα. 

Αυτό θα μπορούσε να οφείλεται είτε στο ότι το μέσο κάνει πολλά posts για το ίδιο άρθρο είτε στο ότι έχει γίνει λάθος στο merge/join των datasets από την in_news.

In [74]:
# μοναδικά άρθρα για τα οποία υπάρχουν posts από λογαριασμούς των ίδιων των sites (source_s == sitename)
print('nbr of unique article links for which there are posts from site social accounts:',
      len(high_comp_posts_from_sites_only_df.external_link_s.unique()))
# σύνολο posts από λογαριασμούς των ίδιων των sites 
print("nbr of posts from sites' social accounts:",
      len(high_comp_posts_from_sites_only_df))

nbr of unique article links for which there are posts from site social accounts: 566489
nbr of posts from sites' social accounts: 769684


In [75]:
# μοναδικά άρθρα για τα οποία υπάρχουν fb posts από λογαριασμό των ίδιων των sites
print("nbr of unique article links for which there are fb posts from site's fb account:",
      len(high_sites_fb_df.external_link_s.unique()))
# σύνολο fb posts από λογαριασμούς των ίδιων των sites 
print("nbr of fb posts from sites' social accounts:",
      len(high_sites_fb_df))

nbr of unique article links for which there are fb posts from site's fb account: 212546
nbr of fb posts from sites' social accounts: 260861


In [76]:
# μοναδικά άρθρα για τα οποία υπάρχουν tweets από το ίδιο το site
print("nbr of unique article links for which there are tweets from site's twitter account:",
      len(high_sites_tw_df.external_link_s.unique()))
# σύνολο tweets από λογαριασμούς των ίδιων των sites 
print("nbr of tweets from sites' social accounts:",
      len(high_sites_tw_df))

nbr of unique article links for which there are tweets from site's twitter account: 420499
nbr of tweets from sites' social accounts: 508823


In [77]:
# μοναδικά link των posts (url_s) για τα οποία υπάρχουν fb posts από λογαριασμό των ίδιων των sites
print("nbr of unique post links for posts from site's social accounts:",
      len(high_comp_posts_from_sites_only_df.url_s.unique()))

nbr of unique post links for posts from site's social accounts: 769684


**Notes:** Από τα παραπάνω προκύπτει ότι υπάρχουν περισσότερα από ένα posts από (υποτίθεται) το ίδιο account για το ίδιο άρθρο. Τα οποία όμως έχουν διαφορετικό link, άρα δεν είναι duplicates. Είναι διαφορετικά posts (υποτίθεται) από λογαριασμό του ίδιου του site για το ίδιο άρθρο. Άρα δεν υπάρχει αντιστοίχιση: ένα fb post ή/και ένα tweet για κάθε άρθρο. Το site μπορεί να έχει κάνει post για ένα άρθρο πολλές φορές ή μπορεί να είναι λάθος οι εγγραφές στο dataset (ίσως από το merge που έκαναν στην innews).

In [78]:
high_sites_tw_df.groupby('url_s').in_id.count().sort_values()

url_s
https://twitter.com/24sports/status/684067047806877696/         1
https://twitter.com/paraskhnio/status/739896633966624773/       1
https://twitter.com/paraskhnio/status/739893257342222338/       1
https://twitter.com/paraskhnio/status/739891852573331458/       1
https://twitter.com/paraskhnio/status/739890429668294657/       1
                                                               ..
https://twitter.com/antinewsgr/status/748914829218328576/       1
https://twitter.com/antinewsgr/status/748853432900018176/       1
https://twitter.com/antinewsgr/status/748842131247292420/       1
https://twitter.com/antinewsgr/status/748944820920430592/       1
https://twitter.com/zougla_online/status/815270832201170948/    1
Name: in_id, Length: 508823, dtype: int64

In [79]:
high_sites_fb_df.groupby('url_s').in_id.count().sort_values()

url_s
https://www.facebook.com/102175189751/posts/10153999911839752      1
https://www.facebook.com/253983294639675/posts/1129751760396153    1
https://www.facebook.com/253983294639675/posts/1129761500395179    1
https://www.facebook.com/253983294639675/posts/1129770033727659    1
https://www.facebook.com/253983294639675/posts/1129781747059821    1
                                                                  ..
https://www.facebook.com/150021178363878/posts/1332659690100015    1
https://www.facebook.com/150021178363878/posts/1332668000099184    1
https://www.facebook.com/150021178363878/posts/1332689486763702    1
https://www.facebook.com/150021178363878/posts/1332652856767365    1
https://www.facebook.com/91157111480/posts/10154009428916481       1
Name: in_id, Length: 260861, dtype: int64

**Notes:** Τα url_s είναι όντως μοναδικά, δηλαδή τα posts είναι διαφορετικά.

In [80]:
high_sites_fb_df.groupby('external_link_s').in_id.count().sort_values()

external_link_s
http://aitoloakarnanianews.gr/%c2%a8%cf%86%ce%b9%ce%ac%cf%83%ce%ba%ce%bf%c2%a8-%ce%b7-%cf%83%cf%85%ce%bc%cf%86%cf%89%ce%bd%ce%af%ce%b1-%ce%b3%ce%b9%ce%b1-%cf%84%ce%b7%ce%bd-%cf%80%cf%81%ce%bf%cf%83%cf%84%ce%b1%cf%83%ce%af%ce%b1/      1
http://www.lifo.gr/now/politics/116443                                                                                                                                                                                                    1
http://www.lifo.gr/now/politics/116510                                                                                                                                                                                                    1
http://www.lifo.gr/now/politics/116573                                                                                                                                                                                                    1
http://www.lifo.gr/now/politics/116630  

**Notes:** Υπάρχουν π.χ. 109 διαφορετικά fb posts για το ίδιο άρθρο του enikos.gr (υποτίθεται) στο facebbok του enikos.

In [81]:
high_sites_fb_df[high_sites_fb_df.external_link_s == 'http://www.enikos.gr/media/413590,Kyriakh-sth-Realnews-Bivlio-me-glyka-gia-na-entypwsiasete--10X5000.html']

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
1295620,1295620,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-10T19:33:45Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1295636,1295636,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-10T18:29:46Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1295782,1295782,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-11T02:01:53Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1295790,1295790,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-11T01:01:01Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1295797,1295797,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-11T00:03:53Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299501,1299501,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-15T18:06:25Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1299513,1299513,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-15T16:35:45Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1299523,1299523,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-15T15:20:41Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0
1299650,1299650,facebook,https://www.facebook.com/303605699676942/posts...,"http://www.enikos.gr/media/413590,Kyriakh-sth-...",enikos.gr,2016-10-15T21:22:21Z,Κυριακή στη Realnews: Βιβλίο με γλυκά για να ε...,0.0,132696.0,0.0,...,enikos,enikos,109,0,40.0,26.0,1.0,94350910.0,1852128.0,6229876.0


In [82]:
high_sites_fb_df[high_sites_fb_df.external_link_s == 'http://www.enikos.gr/media/413590,Kyriakh-sth-Realnews-Bivlio-me-glyka-gia-na-entypwsiasete--10X5000.html'].full_artcl_text

1295620     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1295636     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1295782     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1295790     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1295797     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
                                 ...                        
1299501     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1299513     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1299523     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1299650     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
1299659     Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα...
Name: full_artcl_text, Length: 109, dtype: object

In [83]:
print(high_sites_fb_df.loc[1295620, 'full_artcl_text'])
print(high_sites_fb_df.loc[1295636, 'full_artcl_text'])

 Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα Ελληνικά το κορυφαίο βιβλίο ζαχαροπλαστικής με 100 κλασικές και μοναδικές συνταγές για γλυκά, από τον βραβευμένο ζαχαροπλάστη Κρίστιαν Χίμπς .  Δείτε εδώ το εξώφυλλο:   50.000€ 10 τυχεροί αναγνώστες κερδίζουν από 5.000€ Ισχύει και για την έκδοση των 2€  Δείτε το τηλεοπτικό σποτ εδώ:    
 Γλυκά για να εντυπωσιάσετε Για πρώτη φορά στα Ελληνικά το κορυφαίο βιβλίο ζαχαροπλαστικής με 100 κλασικές και μοναδικές συνταγές για γλυκά, από τον βραβευμένο ζαχαροπλάστη Κρίστιαν Χίμπς .  Δείτε εδώ το εξώφυλλο:   50.000€ 10 τυχεροί αναγνώστες κερδίζουν από 5.000€ Ισχύει και για την έκδοση των 2€  Δείτε το τηλεοπτικό σποτ εδώ:    


**Notes:** το full_artcl_text είναι το ίδιο. Άρα υπάρχουν 109 διαφορετικές αναρτήσεις στο fb του enikos για το ίδιο άρθρο.

In [84]:
high_sites_fb_df[high_sites_fb_df.full_artcl_text.duplicated()][['text_txt_el', 'full_artcl_text']]

,text_txt_el,full_artcl_text
35756,Αστέρας του ΝΒΑ «γλεντάει» πιτσιρικά σε μπασκε...,Θα το θυμάται για πολλά χρόνια. Ο 11χρονος κ...
37707,Ουαλία - Βέλγιο 0-0 (LIVE) Στο Λιλ απόψε (22:0...,Στο Λιλ απόψε (22:00) ο δεύτερος προημιτελικό...
40602,Οι βουτιές του Αδωνι και της Ευγενίας στην Μύκ...,Στη Μύκονο βρέθηκαν τις προηγούμενες ημέρες ο...
40658,ΗΠΑ: Το υπουργείο Εξωτερικών ανοίγει εκ νέου έ...,Το υπουργείο Εξωτερικών των ΗΠΑ ανακοίνωσε ότ...
41126,Εγινε ρεζίλι: Η απομυθοποίηση της Καρντάσιαν- ...,Έγινε διάσημη για ένα κυρίως πράγμα... για τι...
...,...,...
2978071,Χιόνια: Η συγκλονιστική φωτογραφία του σταυρού...,Είναι από τις φωτογραφίες που δεν θέλουν πολλ...
2978072,Η Πέγκυ Ζήνα μίλησε για το πρόβλημα υγείας της...,Η αφοπλιστικά ειλικρινής εξομολόγηση της τραγ...
2978073,Τραγούδησε Βίσση στο Γεωργία έχεις ταλέντο κι ...,Δείτε την εξαιρετική ερμηνεία της! \r\n\t\t\t...
2978121,Απίστευτη αποκάλυψη για τον George Michael! Ήθ...,Τραγικό παιχνίδι της μοίρας για τον George Mi...


In [85]:
high_sites_fb_df.nunique()

in_id              260861
type_s                  1
url_s              260861
external_link_s    212546
source_s               35
timestamp_dt       251255
text_txt_el        227809
retweet_count_l         1
followers_i           245
following_i             1
mentions_i              1
comments_i            259
likes_i              1855
full_artcl_text    209675
sitename               41
source_s_new           41
facebook_posts         73
tweets                  1
retweets_total        341
likes_total          1946
comments_total        277
followers_total     44054
following_total     16224
mentions_total       3578
dtype: int64

**Notes:** υπάρχουν 260.861 διαφορετικά fb posts, από 41 διαφορετικά sites, για 212.546 διαφορετικά άρθρα. θα μπορούσαμε να πετάξουμε τα duplicate άρθρα (external_link_s)

**Remove posts with duplicate values in external_link_s column**

**(keep one post for each article)**

In [86]:
high_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)
print('fb')
print(high_sites_fb_df.nunique(), '\r')

high_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)
print('twitter')
print(high_sites_tw_df.nunique())

/tmp/ipykernel_629681/1703266885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)


fb
in_id              212546
type_s                  1
url_s              212546
external_link_s    212546
source_s               35
timestamp_dt       206962
text_txt_el        211646
retweet_count_l         1
followers_i           245
following_i             1
mentions_i              1
comments_i            259
likes_i              1829
full_artcl_text    209675
sitename               41
source_s_new           41
facebook_posts         73
tweets                  1
retweets_total        341
likes_total          1946
comments_total        277
followers_total     44054
following_total     16224
mentions_total       3578
dtype: int64 
twitter


/tmp/ipykernel_629681/1703266885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)


in_id              420499
type_s                  1
url_s              420499
external_link_s    420499
source_s               71
timestamp_dt       415728
text_txt_el        420496
retweet_count_l        60
followers_i           460
following_i           106
mentions_i             33
comments_i              1
likes_i                 1
full_artcl_text    416335
sitename               63
source_s_new           63
facebook_posts          1
tweets                187
retweets_total       1039
likes_total           894
comments_total        205
followers_total     90630
following_total     30136
mentions_total       8575
dtype: int64


**Remove source_s column**

In [87]:
high_sites_fb_df.drop('source_s', axis=1, inplace=True)
high_sites_tw_df.drop('source_s', axis=1, inplace=True)

/tmp/ipykernel_629681/3005234146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_sites_fb_df.drop('source_s', axis=1, inplace=True)
/tmp/ipykernel_629681/3005234146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_sites_tw_df.drop('source_s', axis=1, inplace=True)


**Calculation based on external_link_s in order to check merge**

In [88]:
high_fb_posts_df = pd.DataFrame()
high_fb_posts_df['article_url'] = high_sites_fb_df['external_link_s']
high_tw_posts_df = pd.DataFrame()
high_tw_posts_df['article_url'] = high_sites_tw_df['external_link_s']
high_all_posts_df = pd.concat([high_fb_posts_df, high_tw_posts_df])

In [89]:
print('high all posts:', len(high_all_posts_df))
print('high unique all posts:', high_all_posts_df.article_url.nunique())

high all posts: 633045
high unique all posts: 566489


**Notes:** 
- υπάρχουν 633.045 posts για 566.489 άρθρα
- υπάρχουν 66.556 άρθρα για τα οποία έχουν γίνει και fb posts και tweets
- το lenght του merged df θα πρέπει να είναι 566.489


**Notes:**

Τα columns των dfs που πρέπει να γίνουν merged είναι:
- in_id: το id του post στο αρχικό comp_df
- type_s: facebook / twitter
- url_s: το link του post
- external_link_s: το link του άρθρου
- timestamp_dt: η ημερομηνία δημοσίευσης του post
- text_txt_el: το κείμενο του post
- retweet_count_l: o αριθμός retweets του tweet
- followers_i: ο αριθμός ακολούθων του λογαριασμού twitter του site
- following_i: ο αριθμός ακολουθούμενων του λογαριασμού twitter του site
- mentions_i: ο αριθμός αναφερόμενων στο tweet
- comments_i: ο αριθμός σχολίων (αφορά το fb)
- likes_i: ο αριθμός likes στο fb post
- full_artcl_text: το κείμενο του άρθρου
- sitename: το όνομα του site
- source_s_new: το όνομα του λογαριασμού στο fb ή στο twitter
- facebook_posts: ο συνολικός αριθμός δημοσιεύσεων στο fb για το κάθε άρθρο από όλους τους χρήστες
- tweets: ο συνολικός αριθμός δημοσιεύσεων στο twitter για το κάθε άρθρο από όλους τους χρήστες
- retweets_total: ο συνολικός αριθμός retweets για το κάθε άρθρο από τα tweets όλων των χρηστών
- likes_total: ο συνολικός αριθμός fb likes για το κάθε άρθρο από τα fb posts όλων των χρηστών
- comments_total: ο συνολικός αριθμός σχολίων για το κάθε άρθρο από τα posts όλων των χρηστών (αφορά το fb) 
- followers_total: o συνολικός αριθμός ακολούθων όλων των χρηστών που έχουν κάνει tweet για το άρθρο
- following_total: o συνολικός αριθμός ακολουθούμενων από όλους τους χρήστες που έχουν κάνει tweet για το άρθρο
- mentions_total: ο συνολικός αριθμός αναφερόμενων στα tweets όλων των χρηστών που έχουν κάνει tweet για το άρθρο 

**Notes2:**
- θα κατασκευαστεί ένα df που θα αποτελείται από όλα τα άρθρα και θα περιέχει τα posts (στο fb ή/και στο twitter)
- υπάρχουν metrics που αφορούν το εκάστοτε συγκεκριμένο άρθρο και το/τα posts που έγιναν για αυτό και metrics που αφορούν το account του site στο social

Άρα, στο dataset με τα posts μόνο από λογαριασμό του ίδιου του μέσου, έχουμε για κάθε άρθρο:
- το site που το δημοσιεύει (sitename)
- το link του άρθρου (external_link_s)
- το κείμενο του άρθρου (full_artcl_text)
- για το fb post:
 - το id του fb post στο αρχικό comp_df (in_id)
 - το link του fb post (url_s)
 - το κείμενο του fb post (text_txt_el)
 - την ημέρα και ώρα δημοσίευσης του fb post (timestamp_dt)
 - τον αριθμό likes του fb post (likes_i)
 - τον αριθμό σχολίων στο fb post (comments_i)
 - το συνολικό αριθμό των fb posts για το άρθρο από όλους τους χρήστες (facebook_posts)
 - το συνολικό αριθμό των likes σε όλα τα fb posts για το άρθρο από όλους τους χρήστες (likes_total) 
 - το συνολικό αριθμό των comments σε όλα τα fb posts για το άρθρο από όλους τους χρήστες (comments_total) 
- για το tweet:
 - το id του tweet στο αρχικό comp_df (in_id)
 - το link του tweet (url_s)
 - το κείμενο του tweet (text_txt_el)
 - την ημέρα και ώρα δημοσίευσης του tweet (timestamp_dt)
 - τον αριθμό retweets του tweet (retweet_count_l)
 - τον αριθμό mentions στο tweet (mentions_i)
 - το συνολικό αριθμό των tweets για το άρθρο από όλους τους χρήστες (tweets)
 - το συνολικό αριθμό retweets όλων των tweets για το άρθρο από όλους τους χρήστες (retweets_total)
 - το συνολικό αριθμό mentions σε όλα τα tweets για το άρθρο από όλους τους χρήστες (mentions_total)
- για το λογαριασμό του site στο social:
 - τον αριθμό των followers στο twitter του site (followers_i)
 - τον αριθμό των following στο twitter του site (following_i)
- για όλους τους λογαριασμόυς στα social, όλων των χρηστών:
 - το συνολικό αριθμό όλων των followers σε όλους τους λογαρισμούς όλων των χρηστών που έκαναν tweet για το άρθρο
 - το συνολικό αριθμό όλων των following όλων των λογαριασμών όλων των χρηστών που έκαναν tweet για το άρθρο

**Notes3:**

Πριν γίνει το merge των high_sites_fb_df και high_sites_tw_df, Θα μετονομάσω κάποια columns, κάποια θα τα αφαιρέσω και όσα απομείνουν θα τους αλλάξω τη σειρά:

Στο high_sites_fb_df θα μείνουν τα παρακάτω columns με τα εξής νέα ονόματα και την εξής σειρά:
- in_id > fb_in_id
- url_s > fb_url
- external_link_s > article_url (KEY)
- timestamp_dt > fb_datetime
- text_txt_el > fb_text
- comments_i > fb_comments
- likes_i > fb_likes
- full_artcl_text > article_text
- sitename > sitename
- facebook_posts > total_fb_posts
- likes_total > total_fb_likes
- comments_total > total_fb_comments

H σειρά θα είναι: `[['sitename', 'article_url', 'article_text', 'fb_in_id', 'fb_url', 'fb_text', 'fb_datetime', 'fb_likes', 'fb_comments', 'total_fb_posts', 'total_fb_likes', 'total_fb_comments']]`

Στο high_sites_tw_df θα μείνουν τα παρακάτω columns με τα εξής νέα ονόματα και την εξής σειρά:
- in_id > tw_in_id
- url_s > tw_url
- external_link_s > article_url (KEY)
- timestamp_dt > tw_datetime
- text_txt_el > tw_text
- retweet_count_l > tw_retweets
- followers_i > site_tw_followers
- following_i > site_tw_following
- mentions_i > tw_mentions
- tweets > total_tweets
- retweets_total > total_retweets
- followers_total > total_followers
- following_total > total_following
- mentions_total > total_mentions

H σειρά θα είναι: `[['article_url', 'tw_in_id', 'tw_url', 'tw_text', 'tw_datetime', 'tw_retweets', 'tw_mentions', 'total_tweets', 'total_retweets', 'total_mentions', 'site_tw_followers', 'site_tw_following', 'total_followers', 'total_following']]`

Το merge θα γίνει με κλειδί το 'article_url' και με how='outer' (θα υπάρχουν NaN values)

**Rename columns and Create new dfs**

In [90]:
# facebook posts
high_sites_fb_df_new = high_sites_fb_df.rename(columns = {'in_id':'fb_in_id',
                                                       'url_s':'fb_url',
                                                      'external_link_s':'article_url',
                                                      'timestamp_dt':'fb_datetime',
                                                       'text_txt_el':'fb_text',
                                                       'comments_i':'fb_comments',
                                                       'likes_i':'fb_likes',
                                                       'full_artcl_text':'article_text',
                                                       'sitename':'site',
                                                       'facebook_posts':'total_fb_posts',
                                                       'likes_total':'total_fb_likes',
                                                       'comments_total':'total_fb_comments',
                                                      })

high_sites_fb_df_new = high_sites_fb_df_new[['site', 'article_url', 'article_text',
                                       'fb_in_id', 'fb_url', 'fb_text', 'fb_datetime',
                                       'fb_likes', 'fb_comments', 'total_fb_posts',
                                       'total_fb_likes', 'total_fb_comments']]

high_sites_fb_df_new.head(1)

,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,total_fb_likes,total_fb_comments
20112,iefimerida,http://www.iefimerida.gr/news/253267/oi-prosfy...,Αναστάτωση προκλήθηκε στο δυτικό αεροδρόμιο σ...,20112,https://www.facebook.com/106952522717824/posts...,Οι πρόσφυγες εγκαταλείπουν το Ελληνικό -Εκλεισ...,2016-02-26T12:52:00Z,0.0,0.0,1,0.0,0.0


In [91]:
# twitter
high_sites_tw_df_new = high_sites_tw_df.rename(columns = {'in_id':'tw_in_id',
                                                          'url_s':'tw_url',
                                                          'external_link_s':'article_url',
                                                          'timestamp_dt':'tw_datetime',
                                                          'text_txt_el':'tw_text',
                                                          'retweet_count_l':'tw_retweets',
                                                          'followers_i':'site_tw_followers',
                                                          'following_i':'site_tw_following',
                                                          'mentions_i':'tw_mentions',
                                                          'tweets':'total_tweets',
                                                          'retweets_total':'total_retweets',
                                                          'followers_total':'total_followers',
                                                          'following_total':'total_following',
                                                          'mentions_total':'total_mentions'
                                                         })

high_sites_tw_df_new = high_sites_tw_df_new[['article_url', 'tw_in_id', 'tw_url', 'tw_text',
                                             'tw_datetime', 'tw_retweets', 'tw_mentions',
                                             'total_tweets', 'total_retweets', 'total_mentions',
                                             'site_tw_followers', 'site_tw_following',
                                             'total_followers', 'total_following']]

high_sites_tw_df_new.head(1)

,article_url,tw_in_id,tw_url,tw_text,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
16587,http://www.iefimerida.gr/news/411111/steit-nti...,16587,https://twitter.com/kathimerini_gr/status/6837...,#Kopinion Ας μιλήσουμε για τον καιρό https://t...,2016-01-03T19:30:11Z,0.0,7822.0,9,163.0,7838.0,494912.0,432.0,532333.0,11693.0


**Create merged df**

In [193]:
high_sites_merged_df = pd.merge(high_sites_fb_df_new, high_sites_tw_df_new, how='outer', on="article_url")
high_sites_merged_df.reset_index(drop=True, inplace=True)
print(len(high_sites_merged_df))
high_sites_merged_df.head(1)

566489


,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/253267/oi-prosfy...,Αναστάτωση προκλήθηκε στο δυτικό αεροδρόμιο σ...,20112.0,https://www.facebook.com/106952522717824/posts...,Οι πρόσφυγες εγκαταλείπουν το Ελληνικό -Εκλεισ...,2016-02-26T12:52:00Z,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
high_sites_merged_df.isna().sum()

site                 353943
article_url               0
article_text         353943
fb_in_id             353943
fb_url               353943
fb_text              353943
fb_datetime          353943
fb_likes             353943
fb_comments          353943
total_fb_posts       353943
total_fb_likes       353943
total_fb_comments    353943
tw_in_id             145990
tw_url               145990
tw_text              145990
tw_datetime          145990
tw_retweets          145990
tw_mentions          145990
total_tweets         145990
total_retweets       145990
total_mentions       145990
site_tw_followers    145990
site_tw_following    145990
total_followers      145990
total_following      145990
dtype: int64

In [195]:
high_sites_merged_df.loc[0, 'article_url']

'http://www.iefimerida.gr/news/253267/oi-prosfyges-egkataleipoyn-elliniko-ekleisan-tin-paraliaki-kai-pane-eidomeni'

## **Low quality**

In [94]:
low_comp_df.dtypes

in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l    float64
followers_i        float64
following_i        float64
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object

In [95]:
#replace whitespace values with nan
low_comp_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [96]:
# unique values
print(low_comp_df.nunique())

in_id              2050824
type_s                   9
url_s              2050755
external_link_s    1031705
source_s             53803
timestamp_dt       1641149
text_txt_el        1504296
retweet_count_l       4703
followers_i          16112
following_i           7034
mentions_i            1557
comments_i             295
likes_i               2561
full_artcl_text     607563
site                   113
dtype: int64


**type_s**

In [97]:
# find rows with type_s values other than 'facebook' and 'twitter'
low_comp_df[(low_comp_df.type_s != 'facebook') & (low_comp_df.type_s != 'twitter')]

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
238084,238084,http://national-pride.org/2016/06/03/%ce%b4%ce...,προκειμένου να μπορέσει η Πυροσβεστική να σβή...,0,4412,0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,national-pride.org
944973,944973,\t\t\t\t\t\t\t\t\t\t\t\t\t— Dolo Jara (@DoloJa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944974,944974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944975,944975,\t\t\t\t\t\t\t\t\t\t\t\t ΔΙΑΒΑΣΤΕ ΑΚΟΜΗ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944976,944976,\t\t\t\t\t\t\t\t\t\t\t\t ΣΟΚ! Σταματάει ο Μέσ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944977,944977,\t\t\t\t\t\t\t\t\t\t\t\t Το χαμένο πέναλτι το...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944978,944978,\t\t\t\t\t\t\t\t\t\t\t\t Τα δάκρυα του Μέσι,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944979,944979,\t\t\t\t\t\t\t\t\t\t\t\t Αργεντινή - Χιλή 2-4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944984,944984,\t\t\t\t\t\t\t\t\t\t\t\t\t— Dolo Jara (@DoloJa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr
944985,944985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contra.gr


In [98]:
# keep rows with type_s values 'facebook' or 'twitter' only
low_comp_df = low_comp_df[(low_comp_df.type_s == 'facebook') | (low_comp_df.type_s == 'twitter')]

In [99]:
# unique values
print(low_comp_df.nunique())

in_id              2050809
type_s                   2
url_s              2050754
external_link_s    1031704
source_s             53802
timestamp_dt       1641148
text_txt_el        1504295
retweet_count_l       4703
followers_i          16112
following_i           7034
mentions_i            1557
comments_i             295
likes_i               2561
full_artcl_text     607563
site                   113
dtype: int64


**NaN values**

In [100]:
print(low_comp_df.isna().sum())

in_id                   0
type_s                  0
url_s                   0
external_link_s         0
source_s                0
timestamp_dt            0
text_txt_el           357
retweet_count_l         0
followers_i             0
following_i             0
mentions_i              0
comments_i              0
likes_i                 0
full_artcl_text    655106
site                    0
dtype: int64


In [101]:
# remove rows with nan values in any column
low_comp_df = low_comp_df.dropna(how='any')

In [102]:
print(low_comp_df.isna().sum())

in_id              0
type_s             0
url_s              0
external_link_s    0
source_s           0
timestamp_dt       0
text_txt_el        0
retweet_count_l    0
followers_i        0
following_i        0
mentions_i         0
comments_i         0
likes_i            0
full_artcl_text    0
site               0
dtype: int64


In [103]:
print(low_comp_df.nunique())

in_id              1395529
type_s                   2
url_s              1395493
external_link_s     610716
source_s             36064
timestamp_dt       1073821
text_txt_el         932010
retweet_count_l       2424
followers_i          13675
following_i           6408
mentions_i            1304
comments_i             292
likes_i               2535
full_artcl_text     607519
site                    67
dtype: int64


**Examine duplicates in url_s column**

In [104]:
# find duplicates in url_s column
len(low_comp_df['url_s'])-len(low_comp_df['url_s'].drop_duplicates())

36

In [105]:
# ποια είναι τα duplicates;
low_comp_df[low_comp_df.url_s.duplicated()]

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
52483,52483,facebook,https://www.facebook.com/18686421862/posts/101...,http://www.pentapostagma.gr/2016/11/%ce%bf%ce%...,ΧΑΡΑ ΠΑΙΔΙΚΗ,2016-11-13T20:51:24Z,Οι Αμερικανοί πεντηκοστιανοί αναθάρρησαν στις ...,0.0,295185.0,0.0,0.0,0.0,1.0,Το μέλος του δημόσιου επιμελητηρίου και επι...,pentapostagma.gr
52496,52496,facebook,https://www.facebook.com/758143797558315/posts...,http://www.pentapostagma.gr/2016/11/%cf%80%ce%...,Τα Πιο Γαματα status Ειναι Εδω.,2016-11-13T20:35:56Z,Παγκόσμια Ημέρα Διαβήτη: Τί να προσέχετε στη δ...,0.0,24158.0,0.0,0.0,0.0,0.0,Η εγκυμοσύνη μπορεί να έχει διάφορες δυσκολί...,pentapostagma.gr
52543,52543,facebook,https://www.facebook.com/1566956273567449/post...,http://www.pentapostagma.gr/2016/11/%cf%80%ce%...,"Εφημερίδα ''Στόχος """,2016-11-13T22:02:11Z,Πού είναι τα ελληνικά λόμπι σε Ουάσινγκτον και...,0.0,14954.0,0.0,0.0,0.0,1.0,Πώς κατάφερε η Ελλάδα να μην έχει ουσιαστικά...,pentapostagma.gr
52552,52552,facebook,https://www.facebook.com/1566956273567449/post...,http://www.pentapostagma.gr/2016/11/%ce%b5%cf%...,Georgia Bounos,2016-11-14T02:36:52Z,Επιτέλους: ΄Έχουμε Αρχιεπίσκοπο! - Pentapostag...,0.0,14954.0,0.0,0.0,0.0,2.0,Μήν χάσετε τόν ἀποκαλυπτικό Στύλο Ὀρθοδοξίας...,pentapostagma.gr
52558,52558,facebook,https://www.facebook.com/1566956273567449/post...,http://www.pentapostagma.gr/2016/11/%cf%84%ce%...,"Εφημερίδα ''Στόχος """,2016-11-13T21:35:00Z,Τα δέκα πιο μυστήρια πράγματα που βρέθηκαν σε ...,0.0,14954.0,0.0,0.0,0.0,0.0,Δείτε το βίντεο με τα δέκα πιο παράξενα αντι...,pentapostagma.gr
52576,52576,facebook,https://www.facebook.com/726373197454676/posts...,http://www.pentapostagma.gr/2016/11/%cf%80%cf%...,Κρητικό Απόρρητο - UoC,2016-11-14T00:45:19Z,Πρόεδρος Φιλιππίνων: «Αν η ISIS έρθει στη χώρα...,0.0,22714.0,0.0,0.0,0.0,0.0,"Ό, τι μπορεί θα κάνει ο πρόεδρος των Φιλιππί...",pentapostagma.gr
52580,52580,facebook,https://www.facebook.com/726373197454676/posts...,http://www.pentapostagma.gr/2016/11/%cf%80%ce%...,Κρητικό Απόρρητο - UoC,2016-11-14T00:45:12Z,Παγκόσμια Ημέρα Διαβήτη: Τί να προσέχετε στη δ...,0.0,22714.0,0.0,0.0,0.0,0.0,Η εγκυμοσύνη μπορεί να έχει διάφορες δυσκολί...,pentapostagma.gr
52596,52596,facebook,https://www.facebook.com/982006511813560/posts...,http://www.pentapostagma.gr/2016/11/%ce%ba%ce%...,Danai Georgiadou,2016-11-13T18:34:13Z,Καταγράφηκε Τσουνάμι ενός μέτρου στη Νέα Ζηλαν...,0.0,54876.0,0.0,0.0,0.0,0.0,Προειδοποίηση για τσουνάμι εξέδωσε η υπηρεσί...,pentapostagma.gr
52612,52612,facebook,https://www.facebook.com/1456499647933954/post...,http://www.pentapostagma.gr/2016/11/%ce%ba%cf%...,Ρόμπερτ Μάκ ΠΑΟΚ,2016-11-13T23:48:50Z,Κυπριακό: Τί συμφώνησαν Ακιντζί και Αναστασιάδ...,0.0,7564.0,0.0,0.0,0.0,0.0,Σε αποκαλύψεις για τα όσα συμφωνήθηκαν ανάμε...,pentapostagma.gr
52616,52616,facebook,https://www.facebook.com/1456499647933954/post...,http://www.pentapostagma.gr/2016/11/%cf%80%ce%...,Κωνσταντινος Μαρινος,2016-11-13T23:47:40Z,Πίεση από τη ΝΔ για οικονομία και απογοητευμέν...,0.0,7564.0,0.0,0.0,0.0,0.0,Με τη ΝΔ να προσμετρά μία ακόμα νίκη επί της...,pentapostagma.gr


**Drop rows with duplicate values in url_s column**

In [106]:
low_comp_df.drop_duplicates('url_s', keep=False, inplace=True)

In [107]:
print(low_comp_df.nunique())

in_id              1395462
type_s                   2
url_s              1395462
external_link_s     610710
source_s             36013
timestamp_dt       1073759
text_txt_el         931987
retweet_count_l       2424
followers_i          13669
following_i           6408
mentions_i            1304
comments_i             292
likes_i               2535
full_artcl_text     607513
site                    67
dtype: int64


**Examine duplicates in full_artcl_text column**

In [108]:
# ποια είναι τα full_artcl_text με τις μεγαλύτερες επαναλήψεις;
low_comp_df.groupby('full_artcl_text').size().sort_values().tail(1)

full_artcl_text
   ©2015 Pentapostagma.gr -  Όροι Χρήσης    409
dtype: int64

In [109]:
# ποια είναι τα text_txt_el με τις μεγαλύτερες επαναλήψεις;
low_comp_df.groupby('text_txt_el').size().sort_values().tail(1)

text_txt_el
Ερωτοσκόπιο Δευτέρας για όλα τα ζώδια !    1558
dtype: int64

In [110]:
# ποια είναι τα timestamp_dt με τις μεγαλύτερες επαναλήψεις;
low_comp_df.groupby('timestamp_dt').size().sort_values().tail(3)

timestamp_dt
2016-02-29T02:15:13Z    85
2016-04-11T11:38:17Z    90
2016-04-11T21:16:23Z    91
dtype: int64

#### **Report on pre-processed high quality dataset**

In [111]:
print('Low quality sites')
print('-' * len('low quality sites'))
print('\r')

# unique values
print('unique values:')
print(low_comp_df.nunique())
print('\r')

# data types
print('data types:')
print(low_comp_df.dtypes)
print('\r')

Low quality sites
-----------------

unique values:
in_id              1395462
type_s                   2
url_s              1395462
external_link_s     610710
source_s             36013
timestamp_dt       1073759
text_txt_el         931987
retweet_count_l       2424
followers_i          13669
following_i           6408
mentions_i            1304
comments_i             292
likes_i               2535
full_artcl_text     607513
site                    67
dtype: int64

data types:
in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l    float64
followers_i        float64
following_i        float64
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object



In [113]:
low_comp_df.dtypes

in_id                int64
type_s              object
url_s               object
external_link_s     object
source_s            object
timestamp_dt        object
text_txt_el         object
retweet_count_l    float64
followers_i        float64
following_i        float64
mentions_i         float64
comments_i         float64
likes_i            float64
full_artcl_text     object
site                object
dtype: object

### **Indentify site name in source_s (ποια posts έχουν γίνει από το ίδιο το μέσο που δημοσίευσε το άρθρο)**

In [114]:
# facebook and twitter counts
low_comp_df.groupby('type_s').in_id.count()

type_s
facebook    847718
twitter     547744
Name: in_id, dtype: int64

In [115]:
pd.set_option('display.max_colwidth', None)

In [116]:
# Ποια source_s ονόματα σχετίζονται με κάθε site;
print(low_comp_df.groupby('site').source_s.unique())

site
24wro.com.gr          [UNIC NEWS-Νο1 Ειδησεις απο ολο τον κοσμο, GiannisRess, Koukoubagia11, இڿڰۣڰۣ Λιγα Λογια Και Καλα இڿڰۣڰۣ, Είμαστε από τον Πειραιά και Γουστάρουμεεεεε, OPA ΤΡΕΛΟ STYLE, PIREAS CITY, tha'thela na'soun edw, ''Aν ησουν αγαπη θα το ενιωθα'', DRAPETSONA ΔΡΑΠΕΤΣΩΝΑ, piraeuspressgr, PIREAS  ΠΕΙΡΑΙΑΣ, 24wroEu, Piraeus Press, 24ωρο, ΟΙ Πεθερές Τρελάθηκαν, Ο Κύκλος Των Βλαμμένων Ποιητών, Και εμένα μου την σπάει όταν με γράφουν, ~~Αγάπη είναι εσύ~~, Απεχθάνομαι τα δήθεν τυπάκια που πλάθουν όλο ψέματα για τη μίζερη ζωή τους., VIP Μεσίτες Ασφαλίσεων, gkaoubiou, wtodB2seYm6BNA, psalidas, tasos___, vasia_tommo, eleni_raza, PLANITISGI, TDiamantidou, letshugzayn28, Chris_Kaltsas, Xariskapsalis, 1stofNovember1, stud1118, Σε αγαπώ... και γι αυτό σου αφιερώνω..., ΟΛΟΙ ΟΙ ΠΕΙΡΑΙΩΤΕΣ ΣΕ ΜΙΑ ΣΕΛΙΔΑ..., Πουλαμε Τρελα Και Οποιος Θελει Αγοραζει., Ε ΡΕ ΤΙ ΚΟΜΜΑΤΑΡΑ ΑΝΕΒΑΣΑ ΠΑΛΙ.ΠΕΤΑΩ ΛΟΥΛΟΥΔΙΑ ΣΤΗΝ ΟΘΟΝΗ ΜΟΥ, Θα μου δωσεις το msn σου?  Βεβαιως-> Pollaoneirakaneis@xylopita.gr, ΔΡΑΠΕΤΣΩΝΑ

In [117]:
# remove substring after first dot in site names
low_comp_df['sitename'] = low_comp_df['site'].replace(r'\..*', '', regex=True)

In [118]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [119]:
low_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,0.0,0.0,0.0,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,toxwni.gr,toxwni


In [120]:
# lower upper case letters in source_s names
low_comp_df['source_s_lower'] = low_comp_df['source_s'].str.lower()

In [121]:
low_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,0.0,0.0,0.0,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,toxwni.gr,toxwni,_odysseusgr


In [122]:
# find source_s names that start with sitename
print(low_comp_df.sitename.nunique())
print(low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].sitename.nunique())
low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].head(1)

67
52


,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower
2509,2509,twitter,https://twitter.com/toxwni/status/710537435499...,http://www.toxwni.gr/more/item/100069-pantelid...,toxwni,2016-03-17T20:45:00Z,https://t.co/MIxsLiDjcA https://t.co/eJzPSpRu3q,0.0,15246.0,5528.0,0.0,0.0,0.0,«Και τα τρία παιδιά ήταν στον ουρανό του αυτ...,toxwni.gr,toxwni,toxwni


**Notes:** 52 out of 67 sites were identified. But are all matches correct?

In [123]:
pd.set_option('display.max_colwidth', None)
#pd.reset_option('display')

In [124]:
# matches
low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].groupby('sitename').source_s_lower.unique()

sitename
24wro                                                                                                   [24wroeu, 24wro]
agriniopress                                                                                              [agriniopress]
altsantiri                                                                                            [altsantiri.gr αλ]
athensmagazine                                                                                       [athensmagazine.gr]
babyradio                                                                                    [babyradio.gr, babyradiogr]
bankingnews                                                                                              [bankingnewsgr]
boro                                                                                               [boro.gr άννα δρούζα]
contra                                                                                             [contra.gr, contragr]
cretalive              

**Notes:** most matches are correct. Exceptions are for: olympiada

In [125]:
false_matches_low = ['olympiada']

In [126]:
all_matches_low = low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].source_s_lower.unique()
print(list(all_matches_low))

['toxwni', 'tsantiri.gr', 'pentapostagma.gr', 'pentapostagma', 'fanpage news', 'fanpage life', 'fanpage diy', 'fanpage videos', 'cyprusnewseu', 'boro.gr άννα δρούζα', 'topontiki', 'politis_news', 'politis.com.cy - εφημερίδα πολίτης', 'pinnokio', 'pinnokiogr', 'pinnokio.gr', 'lamianow', 'metrosport.gr', 'thecallergr', 'thestival', 'newsitamea', 'press-gr.blogspot.com', 'olympiada', 'olympia.gr', 'newsbomb.gr', 'newsbombgr', 'newsbombcy', 'cretalive.gr', 'cretalive', 'sportdoggr', 'sportdog.gr', 'thebestportal', 'contra.gr', 'contragr', 'oparlapipas.com', 'dialogoscy', 'dialogos', 'altsantiri.gr αλ', 'mignatioucom', 'mignatiou', 'makeleio.gr', 'madata.gr', 'ereportaz', 'tribunegr', 'nooz.gr', 'nooz_gr', 'babyradio.gr', 'babyradiogr', 'e-thessalia.gr - εφημερίδα "η θεσσαλία"', 'e-thessalia.gr - εφημερίδα "θεσσαλία"', 'enallaktikosgr', 'matrix24gr', 'matrix24.gr', 'patrasevents.gr', 'patrasevents_gr', 'cretapost', 'vimaorthodoxias', 'fthisgr', 'karfitsagr', 'triklopodia_gr', 'triklopodia.c

In [127]:
true_matches_low = []
for site in all_matches_low:
    if site not in false_matches_low:
        true_matches_low.append(site)
print(true_matches_low)

['toxwni', 'tsantiri.gr', 'pentapostagma.gr', 'pentapostagma', 'fanpage news', 'fanpage life', 'fanpage diy', 'fanpage videos', 'cyprusnewseu', 'boro.gr άννα δρούζα', 'topontiki', 'politis_news', 'politis.com.cy - εφημερίδα πολίτης', 'pinnokio', 'pinnokiogr', 'pinnokio.gr', 'lamianow', 'metrosport.gr', 'thecallergr', 'thestival', 'newsitamea', 'press-gr.blogspot.com', 'olympia.gr', 'newsbomb.gr', 'newsbombgr', 'newsbombcy', 'cretalive.gr', 'cretalive', 'sportdoggr', 'sportdog.gr', 'thebestportal', 'contra.gr', 'contragr', 'oparlapipas.com', 'dialogoscy', 'dialogos', 'altsantiri.gr αλ', 'mignatioucom', 'mignatiou', 'makeleio.gr', 'madata.gr', 'ereportaz', 'tribunegr', 'nooz.gr', 'nooz_gr', 'babyradio.gr', 'babyradiogr', 'e-thessalia.gr - εφημερίδα "η θεσσαλία"', 'e-thessalia.gr - εφημερίδα "θεσσαλία"', 'enallaktikosgr', 'matrix24gr', 'matrix24.gr', 'patrasevents.gr', 'patrasevents_gr', 'cretapost', 'vimaorthodoxias', 'fthisgr', 'karfitsagr', 'triklopodia_gr', 'triklopodia.com', 'triklop

In [128]:
low_comp_df['source_s_new'] = low_comp_df.apply(lambda x: x['sitename'] if x['source_s_lower'] 
                                                  in true_matches_low else x['source_s_lower'], axis=1)

In [129]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [130]:
low_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site,sitename,source_s_lower,source_s_new
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,0.0,0.0,0.0,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,toxwni.gr,toxwni,_odysseusgr,_odysseusgr


**Posts on social from websites accounts**

In [131]:
low_comp_df[low_comp_df.source_s_new == low_comp_df.sitename].groupby('type_s').in_id.count()

type_s
facebook    197394
twitter     226250
Name: in_id, dtype: int64

### **Calculations for total counts**

**Nbr of posts on social per article**

In [132]:
#pd.set_option('display.max_colwidth', None)
pd.reset_option('display')

/tmp/ipykernel_629681/912013155.py:2: FutureWarning: column_space is deprecated and will be removed in a future version. Use df.to_string(col_space=...) instead.
  pd.reset_option('display')


In [133]:
# count social posts per article
low_comp_df['social_posts'] = low_comp_df.groupby(['external_link_s', 'type_s']).in_id.transform('count')

In [134]:
# write facebook counts in separate column
low_comp_df['facebook_posts'] = low_comp_df.apply(lambda x: 
                                                    x['social_posts'] if x['type_s']=='facebook' else 0, axis=1)

In [135]:
# write tweets counts in separate column
low_comp_df['tweets'] = low_comp_df.apply(lambda x: 
                                                    x['social_posts'] if x['type_s']=='twitter' else 0, axis=1)

**Nbr of retweets per article**

In [136]:
low_comp_df['retweets_total'] = low_comp_df.groupby('external_link_s').retweet_count_l.transform('sum')
low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.retweets_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,likes_i,full_artcl_text,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,...,0.0,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,toxwni.gr,toxwni,_odysseusgr,_odysseusgr,11,0,11,498.0


**Nbr of likes per article**

In [137]:
low_comp_df.columns

Index(['in_id', 'type_s', 'url_s', 'external_link_s', 'source_s',
       'timestamp_dt', 'text_txt_el', 'retweet_count_l', 'followers_i',
       'following_i', 'mentions_i', 'comments_i', 'likes_i', 'full_artcl_text',
       'site', 'sitename', 'source_s_lower', 'source_s_new', 'social_posts',
       'facebook_posts', 'tweets', 'retweets_total'],
      dtype='object')

In [138]:
low_comp_df['likes_total'] = low_comp_df.groupby('external_link_s').likes_i.transform('sum')
low_comp_df[(low_comp_df.type_s=='facebook') & (low_comp_df.likes_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,full_artcl_text,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total
643,643,facebook,https://www.facebook.com/387269487952448/posts...,http://www.toxwni.gr/more/no-comments/item/246...,Το Xωνί,2016-02-07T22:00:00Z,Το ΧΩΝΙ - Οι γυμνές ΦΩΤΟΓΡΑΦΙΕΣ των σταρ που σ...,0.0,328849.0,0.0,...,Οι σούπερ σταρ του Χόλιγουντ ακόμη προσπαθούν...,toxwni.gr,toxwni,το xωνί,το xωνί,2,2,0,0.0,61.0


**Nbr of comments per article**

In [139]:
low_comp_df['comments_total'] = low_comp_df.groupby('external_link_s').comments_i.transform('sum')
low_comp_df[(low_comp_df.type_s=='facebook') & (low_comp_df.comments_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,site,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total
643,643,facebook,https://www.facebook.com/387269487952448/posts...,http://www.toxwni.gr/more/no-comments/item/246...,Το Xωνί,2016-02-07T22:00:00Z,Το ΧΩΝΙ - Οι γυμνές ΦΩΤΟΓΡΑΦΙΕΣ των σταρ που σ...,0.0,328849.0,0.0,...,toxwni.gr,toxwni,το xωνί,το xωνί,2,2,0,0.0,61.0,2.0


**Nbr of followers per article**

In [140]:
low_comp_df['followers_total'] = low_comp_df.groupby('external_link_s').followers_i.transform('sum')
low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.followers_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,...,toxwni,_odysseusgr,_odysseusgr,11,0,11,498.0,0.0,0.0,27918.0


**Nbr of following per article**

In [141]:
low_comp_df['following_total'] = low_comp_df.groupby('external_link_s').following_i.transform('sum')
low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.following_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,source_s_lower,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,...,_odysseusgr,_odysseusgr,11,0,11,498.0,0.0,0.0,27918.0,12173.0


**Nbr of mentions per article**

In [142]:
low_comp_df['mentions_total'] = low_comp_df.groupby('external_link_s').mentions_i.transform('sum')
low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.mentions_total>0)].head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,source_s_new,social_posts,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,_OdysseusGR,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,...,_odysseusgr,11,0,11,498.0,0.0,0.0,27918.0,12173.0,6072.0


**Remove unneeded columns**

In [143]:
low_comp_df.columns

Index(['in_id', 'type_s', 'url_s', 'external_link_s', 'source_s',
       'timestamp_dt', 'text_txt_el', 'retweet_count_l', 'followers_i',
       'following_i', 'mentions_i', 'comments_i', 'likes_i', 'full_artcl_text',
       'site', 'sitename', 'source_s_lower', 'source_s_new', 'social_posts',
       'facebook_posts', 'tweets', 'retweets_total', 'likes_total',
       'comments_total', 'followers_total', 'following_total',
       'mentions_total'],
      dtype='object')

In [144]:
unneeded_cols = ['source_s', 'site', 'source_s_lower', 'social_posts']

In [145]:
low_comp_df.drop(unneeded_cols, axis=1, inplace=True)
low_comp_df.head(1)

,in_id,type_s,url_s,external_link_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
0,0,twitter,https://twitter.com/_OdysseusGR/status/6826106...,http://www.toxwni.gr/ellada/193141-papadimouli...,2015-12-31T19:13:52Z,@2_plays happy new year!,0.0,54.0,94.0,0.0,...,toxwni,_odysseusgr,0,11,498.0,0.0,0.0,27918.0,12173.0,6072.0


In [146]:
len(low_comp_df)

1395462

In [147]:
len(low_comp_df.external_link_s.unique())

610710

### **Make df with posts only from sites accounts**

In [148]:
low_comp_df[low_comp_df.source_s_new == low_comp_df.sitename]

,in_id,type_s,url_s,external_link_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
192,192,twitter,https://twitter.com/sport24/status/68589509764...,http://www.toxwni.gr/ellada/194262-ipo-dialisi...,2016-01-09T20:45:09Z,Επεισόδια στο Προοδευτική-Κερατσίνι https://t....,0.0,106828.0,202.0,5653.0,...,toxwni,toxwni,0,4,43.0,0.0,0.0,249043.0,17753.0,5653.0
282,282,twitter,https://twitter.com/Enallaktikosgr/status/6876...,http://www.toxwni.gr/ellada/194576-pappas-ipox...,2016-01-14T17:03:16Z,Η Ιερά Σύνοδος εξομοιώνει την αποτέφρωση νεκρώ...,0.0,2070.0,426.0,0.0,...,toxwni,toxwni,0,6,8.0,0.0,0.0,5503.0,3094.0,0.0
839,839,twitter,https://twitter.com/thestival/status/702451487...,http://www.toxwni.gr/xoni-apokleistika/198184-...,2016-02-24T11:14:20Z,Ποιες αρρώστιες μεταδίδει ο κάδος του πλυντηρί...,0.0,9577.0,623.0,248.0,...,toxwni,toxwni,0,1,0.0,0.0,0.0,9577.0,623.0,248.0
2328,2328,twitter,https://twitter.com/thestival/status/709838991...,http://www.toxwni.gr/ellada/199586-binteo-%C2%...,2016-03-15T20:29:38Z,"Η μεγάλη επιστροφή της ""Ζήνα"" στην μικρή οθόνη...",0.0,9575.0,623.0,248.0,...,toxwni,toxwni,0,1,0.0,0.0,0.0,9575.0,623.0,248.0
2403,2403,twitter,https://twitter.com/piraeuspressgr/status/7101...,http://www.toxwni.gr/ellada/199668-famellos-th...,2016-03-16T21:50:23Z,ΣΟΚΑΡΟΥΝ ΟΙ ΛΕΠΤΟΜΕΡΕΙΕΣ...\n\nhttps://t.co/ey...,0.0,347.0,186.0,0.0,...,toxwni,toxwni,0,9,1658.0,0.0,0.0,88442.0,84431.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050819,2050819,twitter,https://twitter.com/periodistagr/status/815218...,http://www.periodista.gr/koinwnia/article/3301...,2016-12-31T17:31:14Z,Δείτε το ΒΙΝΤΕΟ από την κάμερα ασφαλείας που π...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050820,2050820,twitter,https://twitter.com/periodistagr/status/815220...,http://www.periodista.gr/politiki/article/3301...,2016-12-31T17:39:50Z,Το Μαξίμου αποκαλύπτει τις βρώμικες σχέσεις τη...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050821,2050821,twitter,https://twitter.com/periodistagr/status/815222...,http://www.periodista.gr/oikonomia/article/330...,2016-12-31T17:44:28Z,Ευ. Τσακαλώτος: Το πλεόνασμα είναι και για τις...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050822,2050822,twitter,https://twitter.com/periodistagr/status/815224...,http://www.periodista.gr/politiki/article/3302...,2016-12-31T17:53:35Z,Η διαπλοκή παίζει τα ρέστα της: Δείτε τι προβά...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0


In [149]:
low_comp_df.loc[2050819, 'full_artcl_text']

' Δείτε το ΒΙΝΤΕΟ Αυτό είναι το βίντεο από τις κάμερες ασφαλείας που οδήγησαν την αστυνομία του Ρίο ντε Τζανέιρο στη διαλεύκανση της δολοφονίας του Έλληνα πρέσβη στη Βραζιλία Κυριάκου Αμοιρίδη. \r\n Στο βίντεο φαίνεται ο 29χρονος συλληφθείς αστυνομικός Sérgio Gomes Moreira Filho να μπαίνει μαζί τον εξάδελφό του στο σπίτι όπου διέμενε ο πρέσβης και η σύζυγός του. \r\n Συγκεκριμένα, σύμφωνα με την εφημερίδα O GLOBO, τη Δευτέρα στις 19:53 ο 29χρονος και ο εξάδελφός του βρίσκονταν για περίπου μια ώρα έξω από το σπίτι. Στη συνέχεια στο βίντεο φαίνεται να μπαίνουν μέσα. Σχεδόν τρεις ώρες αργότερα ο 29χρονος φερόμενος ως δράστης φεύγει μόνος του με το αυτοκίνητο που είχε νοικιάσει ο πρέσβης και στη συνέχεια επιστρέφει πεζός. \r\n Στις 2:30 τα ξημερώματα της Τρίτης ο 29χρονος ξαναπαρκάρει έξω από το σπίτι. Την εμφάνισή του κάνει και ο εξάδελφός του. Οι δύο φαίνονται να κάνουν κάτι στο πίσω κάθισμα του αυτοκινήτου. Οι αρχές λένε πως το αυτοκίνητο αναχώρησε από το σπίτι με το πτώμα του διπλωμάτη

In [150]:
print(low_comp_df.loc[2050819, 'external_link_s'])

http://www.periodista.gr/koinwnia/article/33018/deite-to-binteo-apo-tin-kamera-asfaleias-pou-prodose-tous-dolofonous-tou-ellina-presbi


In [151]:
low_comp_df.loc[2050819, 'text_txt_el']

'Δείτε το ΒΙΝΤΕΟ από την κάμερα ασφαλείας που πρόδωσε τους δολοφόνους του Έλληνα πρέσβη https://t.co/hr0Mrn0neF'

**Create df with posts from sites only**

In [152]:
low_comp_posts_from_sites_only_df = low_comp_df[low_comp_df.source_s_new == low_comp_df.sitename]

In [153]:
low_comp_posts_from_sites_only_df

,in_id,type_s,url_s,external_link_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
192,192,twitter,https://twitter.com/sport24/status/68589509764...,http://www.toxwni.gr/ellada/194262-ipo-dialisi...,2016-01-09T20:45:09Z,Επεισόδια στο Προοδευτική-Κερατσίνι https://t....,0.0,106828.0,202.0,5653.0,...,toxwni,toxwni,0,4,43.0,0.0,0.0,249043.0,17753.0,5653.0
282,282,twitter,https://twitter.com/Enallaktikosgr/status/6876...,http://www.toxwni.gr/ellada/194576-pappas-ipox...,2016-01-14T17:03:16Z,Η Ιερά Σύνοδος εξομοιώνει την αποτέφρωση νεκρώ...,0.0,2070.0,426.0,0.0,...,toxwni,toxwni,0,6,8.0,0.0,0.0,5503.0,3094.0,0.0
839,839,twitter,https://twitter.com/thestival/status/702451487...,http://www.toxwni.gr/xoni-apokleistika/198184-...,2016-02-24T11:14:20Z,Ποιες αρρώστιες μεταδίδει ο κάδος του πλυντηρί...,0.0,9577.0,623.0,248.0,...,toxwni,toxwni,0,1,0.0,0.0,0.0,9577.0,623.0,248.0
2328,2328,twitter,https://twitter.com/thestival/status/709838991...,http://www.toxwni.gr/ellada/199586-binteo-%C2%...,2016-03-15T20:29:38Z,"Η μεγάλη επιστροφή της ""Ζήνα"" στην μικρή οθόνη...",0.0,9575.0,623.0,248.0,...,toxwni,toxwni,0,1,0.0,0.0,0.0,9575.0,623.0,248.0
2403,2403,twitter,https://twitter.com/piraeuspressgr/status/7101...,http://www.toxwni.gr/ellada/199668-famellos-th...,2016-03-16T21:50:23Z,ΣΟΚΑΡΟΥΝ ΟΙ ΛΕΠΤΟΜΕΡΕΙΕΣ...\n\nhttps://t.co/ey...,0.0,347.0,186.0,0.0,...,toxwni,toxwni,0,9,1658.0,0.0,0.0,88442.0,84431.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050819,2050819,twitter,https://twitter.com/periodistagr/status/815218...,http://www.periodista.gr/koinwnia/article/3301...,2016-12-31T17:31:14Z,Δείτε το ΒΙΝΤΕΟ από την κάμερα ασφαλείας που π...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050820,2050820,twitter,https://twitter.com/periodistagr/status/815220...,http://www.periodista.gr/politiki/article/3301...,2016-12-31T17:39:50Z,Το Μαξίμου αποκαλύπτει τις βρώμικες σχέσεις τη...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050821,2050821,twitter,https://twitter.com/periodistagr/status/815222...,http://www.periodista.gr/oikonomia/article/330...,2016-12-31T17:44:28Z,Ευ. Τσακαλώτος: Το πλεόνασμα είναι και για τις...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0
2050822,2050822,twitter,https://twitter.com/periodistagr/status/815224...,http://www.periodista.gr/politiki/article/3302...,2016-12-31T17:53:35Z,Η διαπλοκή παίζει τα ρέστα της: Δείτε τι προβά...,0.0,4565.0,1574.0,0.0,...,periodista,periodista,0,1,0.0,0.0,0.0,4565.0,1574.0,0.0


**From df with posts from sites only, Create one df with fb posts and one for tweets**

In [154]:
low_sites_fb_df = low_comp_posts_from_sites_only_df[low_comp_posts_from_sites_only_df.type_s=='facebook']
print(len(low_sites_fb_df))
low_sites_tw_df = low_comp_posts_from_sites_only_df[low_comp_posts_from_sites_only_df.type_s=='twitter']
print(len(low_sites_tw_df))

197394
226250


In [155]:
low_sites_tw_df.head(1)

,in_id,type_s,url_s,external_link_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
192,192,twitter,https://twitter.com/sport24/status/68589509764...,http://www.toxwni.gr/ellada/194262-ipo-dialisi...,2016-01-09T20:45:09Z,Επεισόδια στο Προοδευτική-Κερατσίνι https://t....,0.0,106828.0,202.0,5653.0,...,toxwni,toxwni,0,4,43.0,0.0,0.0,249043.0,17753.0,5653.0


**Remove posts with duplicate values in external_link_s column**

**(keep one post for each article)**

In [156]:
low_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)
print('fb')
print(low_sites_fb_df.nunique(), '\r')

low_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)
print('twitter')
print(low_sites_tw_df.nunique())

/tmp/ipykernel_629681/892613004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)


fb
in_id              175878
type_s                  1
url_s              175878
external_link_s    175878
timestamp_dt       171946
text_txt_el        175353
retweet_count_l         1
followers_i           179
following_i             1
mentions_i              1
comments_i            198
likes_i              1237
full_artcl_text    174869
sitename               33
source_s_new           33
facebook_posts         51
tweets                  1
retweets_total         94
likes_total          1377
comments_total        210
followers_total     16244
following_total      4131
mentions_total        504
dtype: int64 
twitter


/tmp/ipykernel_629681/892613004.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)


in_id              192584
type_s                  1
url_s              192584
external_link_s    192584
timestamp_dt       191599
text_txt_el        192583
retweet_count_l        39
followers_i           189
following_i            61
mentions_i             21
comments_i              1
likes_i                 1
full_artcl_text    192002
sitename               55
source_s_new           55
facebook_posts          1
tweets                 83
retweets_total        389
likes_total           709
comments_total        216
followers_total     39065
following_total     15234
mentions_total       2486
dtype: int64


**Calculation based on external_link_s in order to check merge**

In [157]:
low_fb_posts_df = pd.DataFrame()
low_fb_posts_df['article_url'] = low_sites_fb_df['external_link_s']
low_tw_posts_df = pd.DataFrame()
low_tw_posts_df['article_url'] = low_sites_tw_df['external_link_s']
low_all_posts_df = pd.concat([low_fb_posts_df, low_tw_posts_df])

In [158]:
print('low all posts:', len(low_all_posts_df))
print('low unique all posts:', low_all_posts_df.article_url.nunique())

low all posts: 368462
low unique all posts: 340067


**Notes:** 
- υπάρχουν 368.462 posts για 340.067 άρθρα
- υπάρχουν 328.395 άρθρα για τα οποία έχουν γίνει και fb posts και tweets
- το lenght του merged df θα πρέπει να είναι 340.067


**Rename columns and Create new dfs**

In [159]:
# facebook posts
low_sites_fb_df_new = low_sites_fb_df.rename(columns = {'in_id':'fb_in_id',
                                                       'url_s':'fb_url',
                                                      'external_link_s':'article_url',
                                                      'timestamp_dt':'fb_datetime',
                                                       'text_txt_el':'fb_text',
                                                       'comments_i':'fb_comments',
                                                       'likes_i':'fb_likes',
                                                       'full_artcl_text':'article_text',
                                                       'sitename':'site',
                                                       'facebook_posts':'total_fb_posts',
                                                       'likes_total':'total_fb_likes',
                                                       'comments_total':'total_fb_comments',
                                                      })

low_sites_fb_df_new = low_sites_fb_df_new[['site', 'article_url', 'article_text',
                                       'fb_in_id', 'fb_url', 'fb_text', 'fb_datetime',
                                       'fb_likes', 'fb_comments', 'total_fb_posts',
                                       'total_fb_likes', 'total_fb_comments']]

low_sites_fb_df_new.head(1)

,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,total_fb_likes,total_fb_comments
23920,tsantiri,http://www.tsantiri.gr/politiki/mparaz-epafon-...,"Θα έχει διαδοχικές συναντήσεις με Παπανδρέου,...",23920,https://www.facebook.com/187426007942451/posts...,Μπαράζ επαφών Τσίπρα για τον εκλογικό νόμο Θα ...,2016-07-18T22:50:44Z,0.0,0.0,1,0.0,0.0


In [160]:
# twitter
low_sites_tw_df_new = low_sites_tw_df.rename(columns = {'in_id':'tw_in_id',
                                                          'url_s':'tw_url',
                                                          'external_link_s':'article_url',
                                                          'timestamp_dt':'tw_datetime',
                                                          'text_txt_el':'tw_text',
                                                          'retweet_count_l':'tw_retweets',
                                                          'followers_i':'site_tw_followers',
                                                          'following_i':'site_tw_following',
                                                          'mentions_i':'tw_mentions',
                                                          'tweets':'total_tweets',
                                                          'retweets_total':'total_retweets',
                                                          'followers_total':'total_followers',
                                                          'following_total':'total_following',
                                                          'mentions_total':'total_mentions'
                                                         })

low_sites_tw_df_new = low_sites_tw_df_new[['article_url', 'tw_in_id', 'tw_url', 'tw_text',
                                             'tw_datetime', 'tw_retweets', 'tw_mentions',
                                             'total_tweets', 'total_retweets', 'total_mentions',
                                             'site_tw_followers', 'site_tw_following',
                                             'total_followers', 'total_following']]

low_sites_tw_df_new.head(1)

,article_url,tw_in_id,tw_url,tw_text,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
192,http://www.toxwni.gr/ellada/194262-ipo-dialisi...,192,https://twitter.com/sport24/status/68589509764...,Επεισόδια στο Προοδευτική-Κερατσίνι https://t....,2016-01-09T20:45:09Z,0.0,5653.0,4,43.0,5653.0,106828.0,202.0,249043.0,17753.0


**Create merged df**

In [161]:
low_sites_merged_df = pd.merge(low_sites_fb_df_new, low_sites_tw_df_new, how='outer', on="article_url")
low_sites_merged_df.reset_index(drop=True, inplace=True)
print(len(low_sites_merged_df))
low_sites_merged_df.head(1)

340067


,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,tsantiri,http://www.tsantiri.gr/politiki/mparaz-epafon-...,"Θα έχει διαδοχικές συναντήσεις με Παπανδρέου,...",23920.0,https://www.facebook.com/187426007942451/posts...,Μπαράζ επαφών Τσίπρα για τον εκλογικό νόμο Θα ...,2016-07-18T22:50:44Z,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Final datasets**

In [162]:
# high_sites_merged_df
# low_sites_merged_df

In [163]:
print('high sites:', high_sites_merged_df.site.nunique())
print('high articles with social posts:', len(high_sites_merged_df))
print('low sites:', low_sites_merged_df.site.nunique())
print('low articles with social posts:', len(low_sites_merged_df))

high sites: 41
high articles with social posts: 566489
low sites: 33
low articles with social posts: 340067


In [164]:
# high articles with both fb post and tweet
high_sites_merged_df[(~high_sites_merged_df.total_fb_posts.isna()) & (~high_sites_merged_df.total_tweets.isna())]

,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
16,iefimerida,http://www.iefimerida.gr/news/253937/kyprios-y...,Ανώτερος λειτουργός Εμπορίου και Βιομηχανίας ...,20865.0,https://www.facebook.com/106952522717824/posts...,Κύπριος υπάλληλος προς τον Πρόεδρο Δημοκρατίας...,2016-03-01T12:31:21Z,2.0,0.0,1.0,...,2016-03-01T10:32:16Z,3.0,25854.0,6.0,15.0,25854.0,170175.0,4250.0,267588.0,8303.0
17,iefimerida,http://www.iefimerida.gr/news/253878/ypothesi-...,"Ο Νίκος Δάβρης, συμφοιτητής του Βαγγέλη Γιακο...",20866.0,https://www.facebook.com/106952522717824/posts...,Υπόθεση Γιακουμάκη: Φρικτές λεπτομέρειες αποκα...,2016-03-01T09:48:54Z,1.0,0.0,1.0,...,2016-03-01T09:49:17Z,1.0,25854.0,2.0,1.0,25854.0,170140.0,4250.0,262842.0,4449.0
18,iefimerida,http://www.iefimerida.gr/news/253748/giati-dak...,O Σιλβέστερ Σταλόνε δάκρυσε ακούγοντας το όνο...,20867.0,https://www.facebook.com/106952522717824/posts...,Γιατί δάκρυσε ο Σταλόνε - Η παράλυση στο πρόσω...,2016-02-29T15:06:18Z,24.0,0.0,1.0,...,2016-02-29T15:06:49Z,1.0,25854.0,2.0,1.0,25854.0,170175.0,4250.0,263032.0,4984.0
39,iefimerida,http://www.iefimerida.gr/news/253995/epeisodia...,Τρομακτικές ήταν οι σκηνές που εκτυλίχθηκαν σ...,21423.0,https://www.facebook.com/106952522717824/posts...,Επεισόδια στο Calais – Γυναίκα πρόσφυγας έκοψε...,2016-03-01T15:00:14Z,1.0,0.0,1.0,...,2016-03-01T15:04:13Z,0.0,25854.0,1.0,0.0,25854.0,170136.0,4250.0,262805.0,4250.0
42,iefimerida,http://www.iefimerida.gr/news/254587/hania-pro...,Αδιαφορώντας για τους ψίθυρους της κλειστής κ...,21454.0,https://www.facebook.com/106952522717824/posts...,Χανιά: Το πρώτο gay σύμφωνο Αδιαφορώντας για τ...,2016-03-04T09:47:33Z,6.0,5.0,1.0,...,2016-03-04T09:50:17Z,0.0,25854.0,1.0,0.0,25854.0,170175.0,4250.0,262844.0,4250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212418,newsit,http://www.newsit.gr/politikh/Pothen-esxes-Ayt...,Τον έχουν τον τρόπο τους δυο από τους κορυφαί...,2977402.0,https://www.facebook.com/170069547711/posts/10...,Πόθεν έσχες: Αυτό είναι το 'χρυσό' ζευγάρι της...,2016-12-28T17:25:04Z,0.0,8.0,1.0,...,2016-12-28T17:26:47Z,0.0,576.0,1.0,0.0,576.0,121252.0,391.0,331248.0,391.0
212420,newsit,http://www.newsit.gr/media/H-Eleni-Menegaki-ai...,"«Να σας το πούμε από τώρα, εμείς δεν θα είμασ...",2977404.0,https://www.facebook.com/170069547711/posts/10...,H Ελένη Μενεγάκη αιφνιδίασε ανακοινώνοντας την...,2016-12-28T16:36:50Z,8.0,3.0,10.0,...,2016-12-28T17:15:35Z,0.0,576.0,1.0,0.0,576.0,121252.0,391.0,1123375.0,391.0
212426,newsit,http://www.newsit.gr/ellada/LOTTO-Klirosi-28-1...,Κλήρωση ΛΟΤΤΟ 1770: Περισσότερα από 300.000 ε...,2977472.0,https://www.facebook.com/170069547711/posts/10...,ΛΟΤΤΟ [Κλήρωση 28/12 - 1770]: Αυτοί είναι οι τ...,2016-12-28T22:25:15Z,1.0,0.0,1.0,...,2016-12-28T22:09:59Z,0.0,576.0,1.0,0.0,576.0,121252.0,391.0,331248.0,391.0
212427,newsit,http://www.newsit.gr/ellada/Dimos-Athinaion-Pa...,- Εξοργιστικό τέλος στο σίριαλ με τη ρόδα παρ...,2977473.0,https://www.facebook.com/170069547711/posts/10...,Δήμος Αθηναίων: Παραιτήθηκε ο Χρήστος Τεντόμας...,2016-12-28T21:44:04Z,5.0,1.0,1.0,...,2016-12-28T21:44:31Z,0.0,576.0,2.0,0.0,576.0,121252.0,391.0,331388.0,748.0


In [165]:
# low articles with both fb post and tweet
low_sites_merged_df[(~low_sites_merged_df.total_fb_posts.isna()) & (~low_sites_merged_df.total_tweets.isna())]

,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
187,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%80%cf%...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24218.0,https://www.facebook.com/114954161944042/posts...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,2016-03-05T13:25:11Z,15.0,0.0,1.0,...,2016-03-05T13:25:20Z,2.0,68.0,3.0,4.0,68.0,8327.0,347.0,66371.0,2692.0
188,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%bc%ce%...,Διατηρεί τη μεγαλύτερη Ιστορική και Θρησκευτ...,24219.0,https://www.facebook.com/114954161944042/posts...,Μιχαήλ Τσίριμπας ο ακούραστος δάσκαλος! - Pent...,2016-03-04T08:50:15Z,2.0,0.0,1.0,...,2016-03-04T06:50:26Z,0.0,68.0,1.0,0.0,68.0,8328.0,347.0,65431.0,347.0
189,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,"Δημοσίευση: 11 Μαρτίου 2016, 11:24 πμ \n ...",24224.0,https://www.facebook.com/114954161944042/posts...,Τέλος και επίσημα ο Σταυρός από τη σημαία! - P...,2016-03-11T11:46:12Z,10.0,4.0,1.0,...,2016-03-11T09:46:24Z,0.0,68.0,2.0,0.0,68.0,8328.0,347.0,66052.0,1789.0
190,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24231.0,https://www.facebook.com/114954161944042/posts...,Το κοινό μυστικό των τριών Ευαγγελίων - Γέροντ...,2016-03-15T14:45:12Z,1.0,0.0,1.0,...,2016-03-15T14:45:23Z,0.0,68.0,1.0,0.0,68.0,8329.0,347.0,65432.0,347.0
191,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%b2%ce%...,Συμπληρώνονται φέτος 195 χρόνια από την Έναρ...,24245.0,https://www.facebook.com/114954161944042/posts...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821 ...,2016-03-25T00:25:11Z,9.0,1.0,1.0,...,2016-03-25T00:25:22Z,1.0,68.0,2.0,2.0,68.0,8328.0,347.0,66232.0,2828.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175871,agriniopress,http://www.agriniopress.gr/%ce%bc%ce%ad%ce%bd%...,Παρά την παγωνιά το Τμήμα Παραδοσιακής Μουσ...,1942866.0,https://www.facebook.com/160422017318141/posts...,"""Μ’ένα ρόδι θα σε προϋπαντήσω"" στην πλατεία Δη...",2016-12-23T20:51:30Z,7.0,0.0,1.0,...,2016-12-23T20:51:31Z,0.0,0.0,1.0,0.0,0.0,1808.0,241.0,24628.0,241.0
175872,agriniopress,http://www.agriniopress.gr/%ce%bc%ce%b5-%ce%b5...,Στην κατάμεστη αίθουσα του Παπαστρατείου Μεγ...,1942867.0,https://www.facebook.com/160422017318141/posts...,Με επιτυχία η χριτουγεννιάτικη εκδήλωση για τη...,2016-12-23T19:49:03Z,1.0,0.0,1.0,...,2016-12-23T19:49:04Z,0.0,0.0,1.0,0.0,0.0,1811.0,242.0,24631.0,242.0
175873,agriniopress,http://www.agriniopress.gr/%cf%80%cf%8e%cf%82-...,Πρώτο ταξίδι του Σπύρου Νικολάου και του Μο...,1942873.0,https://www.facebook.com/160422017318141/posts...,"""Πώς βρεθήκαμε εδώ!"": Η πρώτη δισκογραφική δου...",2016-12-24T10:27:04Z,0.0,0.0,1.0,...,2016-12-24T10:27:05Z,0.0,0.0,1.0,0.0,0.0,1808.0,241.0,24628.0,241.0
175874,agriniopress,http://www.agriniopress.gr/%ce%b7-%cf%81%cf%8c...,Ο Δημοτικός Κινηματογράφος Αγρινίου “Ανεσις”...,1942876.0,https://www.facebook.com/160422017318141/posts...,"""Η Ρόζα της Σμύρνης"" και ""Τραγούδα!"" από την Π...",2016-12-28T13:01:57Z,6.0,0.0,1.0,...,2016-12-28T13:01:58Z,0.0,0.0,1.0,0.0,0.0,1811.0,242.0,24631.0,242.0


## **Articles only**

In [166]:
high_articles_only_df = high_comp_df[['in_id', 'sitename', 'external_link_s', 'full_artcl_text', 'timestamp_dt', 'facebook_posts', 'tweets', 'retweets_total', 'likes_total', 'comments_total']]
low_articles_only_df = low_comp_df[['in_id', 'sitename', 'external_link_s', 'full_artcl_text', 'timestamp_dt', 'facebook_posts', 'tweets', 'retweets_total', 'likes_total', 'comments_total']]


In [167]:
high_articles_only_df.drop_duplicates('full_artcl_text', keep='first', inplace=True)
low_articles_only_df.drop_duplicates('full_artcl_text', keep='first', inplace=True)

/tmp/ipykernel_629681/2498366609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_articles_only_df.drop_duplicates('full_artcl_text', keep='first', inplace=True)
/tmp/ipykernel_629681/2498366609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_articles_only_df.drop_duplicates('full_artcl_text', keep='first', inplace=True)


In [168]:
high_articles_only_df = high_articles_only_df.rename(columns = {'sitename':'site',
                                                         'external_link_s':'article_url',
                                                         'full_artcl_text':'article_text',
                                                         'timestamp_dt':'datetime',
                                                         'facebook_posts':'fb_posts'})
low_articles_only_df = low_articles_only_df.rename(columns = {'sitename':'site',
                                                         'external_link_s':'article_url',
                                                         'full_artcl_text':'article_text',
                                                         'timestamp_dt':'datetime',
                                                         'facebook_posts':'fb_posts'})

In [169]:
high_articles_only_df.head(1)

,in_id,site,article_url,article_text,datetime,fb_posts,tweets,retweets_total,likes_total,comments_total
16430,16430,iefimerida,http://www.iefimerida.gr/news/409912/diadiloti...,Ενα απίστευτο περιστατικό συνέβη σε συγκέντρω...,2016-01-01T00:45:12Z,0,4,19739.0,0.0,0.0


In [170]:
low_articles_only_df.head(1)

,in_id,site,article_url,article_text,datetime,fb_posts,tweets,retweets_total,likes_total,comments_total
0,0,toxwni,http://www.toxwni.gr/ellada/193141-papadimouli...,Σχετικά Άρθρα \n\t\t \n\t\t\t\t\t\t \n\t\t\t\...,2015-12-31T19:13:52Z,0,11,498.0,0.0,0.0


In [171]:
print('high quality articles only df name: high_articles_only_df')
print('high unique articles:', high_articles_only_df.article_url.nunique())
print('high unique sites:', high_articles_only_df.site.nunique())
print('\r')
print('low quality articles only df name: low_articles_only_df')
print('low unique articles:', low_articles_only_df.article_url.nunique())
print('low unique sites:', low_articles_only_df.site.nunique())

high quality articles only df name: high_articles_only_df
high unique articles: 901900
high unique sites: 68

low quality articles only df name: low_articles_only_df
low unique articles: 607513
low unique sites: 67


**Sitenames**

In [172]:
high_articles_only_df.site.unique()

array(['iefimerida', 'tvxs', 'thetoc', 'panathinaikos24', 'stokokkino',
       'newsbeast', 'skai', 'tovima', 'thepressproject', 'lifo',
       'astratv', '24sports', 'ert', 'zougla', 'thepaper', 'antenna',
       'enikonomia', 'koutipandoras', 'athensvoice', 'news',
       'ntokoumenta', 'press724', 'star', 'parapolitika', 'xrimaonline',
       'antinews', 'enikos', 'freepen', 'anatropinews', 'news247', 'cna',
       'balla', 'infokids', 'cyclades24', 'liberal', 'avgi',
       'naftemporiki', 'fnews', 'capital', 'dimokratiki', 'xenesglosses',
       'paraskhnio', 'rodiaki', 'ysterografa', 'efsyn', 'sofokleousin',
       'onvolos', 'theguardian', 'kathimerini', 'amna', 'protagon',
       'eleftherostypos', 'aitoloakarnanianews', 'kontranews',
       'enetpress', '902', 'protothema', 'newspao', 'left', 'gazzetta',
       'aek365', 'taxydromos', 'real', 'bizznews', 'ipaideia', 'newpost',
       'newsit', 'presspublica'], dtype=object)

In [173]:
low_articles_only_df.site.unique()

array(['toxwni', 'tsantiri', 'pentapostagma', 'fanpage', 'cyprusnews',
       'boro', 'topontiki', 'politis', 'pinnokio', 'lamianow',
       'tilestwra', 'metrosport', 'thecaller', 'thestival', 'newsitamea',
       'planet-greece', 'stoxos', 'filathlos', 'diaforetiko', 'aixmi',
       'press-gr', 'olympia', 'newsbomb', 'cretalive', 'piraeusview',
       'sportdog', 'oroskopos', 'thebest', 'ekklisiaonline', 'contra',
       'oparlapipas', 'dialogos', 'altsantiri', 'mignatiou', 'iapopsi',
       'makeleio', 'madata', 'ereportaz', 'tribune', 'nooz', 'babyradio',
       'e-thessalia', 'enallaktikos', 'matrix24', 'patrasevents',
       'cretapost', 'vimaorthodoxias', 'fthis', 'karfitsa', 'videotex',
       'triklopodia', 'i-eidisi', '24wro', 'bankingnews', 'sport-fm',
       'piraeuspress', 'sdna', 'patrisnews', 'apotis4stis5',
       'athensmagazine', 'sport24', 'agriniopress', 'globalview',
       'koolnews', 'voicenews', 'dikaiologitika', 'periodista'],
      dtype=object)

## Export csvs

### High

In [204]:
high_sites_fb_tw_df = high_sites_merged_df[(~high_sites_merged_df.total_fb_posts.isna())\
                                           & (~high_sites_merged_df.total_tweets.isna())].copy()
print(high_sites_merged_df.shape, high_sites_fb_tw_df.shape)

(566489, 25) (66556, 25)


In [205]:
high_sites_fb_tw_df['title'] = \
high_sites_fb_tw_df['tw_text'].str.replace('\n', ' ') \
.str.replace('http[\:a-zA-Z\/\.0-9]*|\#[a-zA-Z]*|\s{2,}', '', regex=True)
high_sites_fb_tw_df.insert(2, 'article_title', high_sites_fb_tw_df.pop('title'))
print(high_sites_fb_tw_df.isna().sum())
high_sites_fb_tw_df.head()

site                 0
article_url          0
article_title        0
article_text         0
fb_in_id             0
fb_url               0
fb_text              0
fb_datetime          0
fb_likes             0
fb_comments          0
total_fb_posts       0
total_fb_likes       0
total_fb_comments    0
tw_in_id             0
tw_url               0
tw_text              0
tw_datetime          0
tw_retweets          0
tw_mentions          0
total_tweets         0
total_retweets       0
total_mentions       0
site_tw_followers    0
site_tw_following    0
total_followers      0
total_following      0
dtype: int64


,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
16,iefimerida,http://www.iefimerida.gr/news/253937/kyprios-y...,Κύπριος υπάλληλος προς τον Πρόεδρο Δημοκρατίας...,Ανώτερος λειτουργός Εμπορίου και Βιομηχανίας ...,20865.0,https://www.facebook.com/106952522717824/posts...,Κύπριος υπάλληλος προς τον Πρόεδρο Δημοκρατίας...,2016-03-01T12:31:21Z,2.0,0.0,...,2016-03-01T10:32:16Z,3.0,25854.0,6.0,15.0,25854.0,170175.0,4250.0,267588.0,8303.0
17,iefimerida,http://www.iefimerida.gr/news/253878/ypothesi-...,Υπόθεση Γιακουμάκη: Φρικτές λεπτομέρειες αποκα...,"Ο Νίκος Δάβρης, συμφοιτητής του Βαγγέλη Γιακο...",20866.0,https://www.facebook.com/106952522717824/posts...,Υπόθεση Γιακουμάκη: Φρικτές λεπτομέρειες αποκα...,2016-03-01T09:48:54Z,1.0,0.0,...,2016-03-01T09:49:17Z,1.0,25854.0,2.0,1.0,25854.0,170140.0,4250.0,262842.0,4449.0
18,iefimerida,http://www.iefimerida.gr/news/253748/giati-dak...,Γιατί δάκρυσε ο Σταλόνε - Η παράλυση στο πρόσω...,O Σιλβέστερ Σταλόνε δάκρυσε ακούγοντας το όνο...,20867.0,https://www.facebook.com/106952522717824/posts...,Γιατί δάκρυσε ο Σταλόνε - Η παράλυση στο πρόσω...,2016-02-29T15:06:18Z,24.0,0.0,...,2016-02-29T15:06:49Z,1.0,25854.0,2.0,1.0,25854.0,170175.0,4250.0,263032.0,4984.0
39,iefimerida,http://www.iefimerida.gr/news/253995/epeisodia...,Επεισόδια στο Calais – Γυναίκα πρόσφυγας έκοψε...,Τρομακτικές ήταν οι σκηνές που εκτυλίχθηκαν σ...,21423.0,https://www.facebook.com/106952522717824/posts...,Επεισόδια στο Calais – Γυναίκα πρόσφυγας έκοψε...,2016-03-01T15:00:14Z,1.0,0.0,...,2016-03-01T15:04:13Z,0.0,25854.0,1.0,0.0,25854.0,170136.0,4250.0,262805.0,4250.0
42,iefimerida,http://www.iefimerida.gr/news/254587/hania-pro...,Χανιά: Το πρώτο gay σύμφωνο,Αδιαφορώντας για τους ψίθυρους της κλειστής κ...,21454.0,https://www.facebook.com/106952522717824/posts...,Χανιά: Το πρώτο gay σύμφωνο Αδιαφορώντας για τ...,2016-03-04T09:47:33Z,6.0,5.0,...,2016-03-04T09:50:17Z,0.0,25854.0,1.0,0.0,25854.0,170175.0,4250.0,262844.0,4250.0


In [213]:
high_sites_fb_tw_df.to_csv('0_csvs_for_IQ_feed/high_sites.csv', index=False)

In [214]:
high_sites_df = pd.read_csv('0_csvs_for_IQ_feed/high_sites.csv', lineterminator='\n')
high_sites_df.head()

,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/253937/kyprios-y...,Κύπριος υπάλληλος προς τον Πρόεδρο Δημοκρατίας...,Ανώτερος λειτουργός Εμπορίου και Βιομηχανίας ...,20865.0,https://www.facebook.com/106952522717824/posts...,Κύπριος υπάλληλος προς τον Πρόεδρο Δημοκρατίας...,2016-03-01T12:31:21Z,2.0,0.0,...,2016-03-01T10:32:16Z,3.0,25854.0,6.0,15.0,25854.0,170175.0,4250.0,267588.0,8303.0
1,iefimerida,http://www.iefimerida.gr/news/253878/ypothesi-...,Υπόθεση Γιακουμάκη: Φρικτές λεπτομέρειες αποκα...,"Ο Νίκος Δάβρης, συμφοιτητής του Βαγγέλη Γιακο...",20866.0,https://www.facebook.com/106952522717824/posts...,Υπόθεση Γιακουμάκη: Φρικτές λεπτομέρειες αποκα...,2016-03-01T09:48:54Z,1.0,0.0,...,2016-03-01T09:49:17Z,1.0,25854.0,2.0,1.0,25854.0,170140.0,4250.0,262842.0,4449.0
2,iefimerida,http://www.iefimerida.gr/news/253748/giati-dak...,Γιατί δάκρυσε ο Σταλόνε - Η παράλυση στο πρόσω...,O Σιλβέστερ Σταλόνε δάκρυσε ακούγοντας το όνο...,20867.0,https://www.facebook.com/106952522717824/posts...,Γιατί δάκρυσε ο Σταλόνε - Η παράλυση στο πρόσω...,2016-02-29T15:06:18Z,24.0,0.0,...,2016-02-29T15:06:49Z,1.0,25854.0,2.0,1.0,25854.0,170175.0,4250.0,263032.0,4984.0
3,iefimerida,http://www.iefimerida.gr/news/253995/epeisodia...,Επεισόδια στο Calais – Γυναίκα πρόσφυγας έκοψε...,Τρομακτικές ήταν οι σκηνές που εκτυλίχθηκαν σ...,21423.0,https://www.facebook.com/106952522717824/posts...,Επεισόδια στο Calais – Γυναίκα πρόσφυγας έκοψε...,2016-03-01T15:00:14Z,1.0,0.0,...,2016-03-01T15:04:13Z,0.0,25854.0,1.0,0.0,25854.0,170136.0,4250.0,262805.0,4250.0
4,iefimerida,http://www.iefimerida.gr/news/254587/hania-pro...,Χανιά: Το πρώτο gay σύμφωνο,Αδιαφορώντας για τους ψίθυρους της κλειστής κ...,21454.0,https://www.facebook.com/106952522717824/posts...,Χανιά: Το πρώτο gay σύμφωνο Αδιαφορώντας για τ...,2016-03-04T09:47:33Z,6.0,5.0,...,2016-03-04T09:50:17Z,0.0,25854.0,1.0,0.0,25854.0,170175.0,4250.0,262844.0,4250.0


In [243]:
print('High quality sites')
print('='*len('High quality sites'))
print('Πλήθος sites:',high_sites_df.site.nunique())
print('Sites list:',high_sites_df.site.unique())
print('Πλήθος άρθρων:',len(high_sites_df))
print('\n'+'-'*70+'\n')
print('High quality sites - Top engagement q75')
print('='*len('High quality sites - Top engagement q75'))
print('Πλήθος sites:',high_top_75_df.site.nunique())
print('Sites list:',high_top_75_df.site.unique())
print('Πλήθος άρθρων:',len(high_top_75_df))
print('\n'+'-'*70+'\n')
print('Low quality sites')
print('='*len('Low quality sites'))
print('Πλήθος sites:',low_sites_df.site.nunique())
print('Sites list:',low_sites_df.site.unique())
print('Πλήθος άρθρων:',len(low_sites_df))
print('\n'+'-'*70+'\n')
print('Low quality sites - Top engagement q66')
print('='*len('Low quality sites - Top engagement q66'))
print('Πλήθος sites:',low_top_66_df.site.nunique())
print('Sites list:',low_top_66_df.site.unique())
print('Πλήθος άρθρων:',len(low_top_66_df))

High quality sites
Πλήθος sites: 28
Sites list: ['iefimerida' 'newsbeast' 'skai' 'tovima' 'astratv' '24sports' 'ert'
 'zougla' 'enikonomia' 'star' 'parapolitika' 'antinews' 'enikos' 'freepen'
 'news247' 'balla' 'infokids' 'cyclades24' 'fnews' 'dimokratiki'
 'protothema' 'newspao' 'left' 'gazzetta' 'aek365' 'real' 'bizznews'
 'newsit']
Πλήθος άρθρων: 66556

----------------------------------------------------------------------

High quality sites - Top engagement q75
Πλήθος sites: 24
Sites list: ['iefimerida' 'newsbeast' 'skai' 'tovima' '24sports' 'zougla' 'star'
 'parapolitika' 'antinews' 'enikos' 'freepen' 'news247' 'balla' 'infokids'
 'cyclades24' 'dimokratiki' 'protothema' 'newspao' 'left' 'gazzetta'
 'aek365' 'real' 'bizznews' 'newsit']
Πλήθος άρθρων: 5011

----------------------------------------------------------------------

Low quality sites
Πλήθος sites: 17
Sites list: ['pentapostagma' 'politis' 'pinnokio' 'newsbomb' 'cretalive' 'sportdog'
 'contra' 'dialogos' 'nooz' 'babyradi

### Low

In [206]:
low_sites_fb_tw_df = low_sites_merged_df[(~low_sites_merged_df.total_fb_posts.isna())\
                                           & (~low_sites_merged_df.total_tweets.isna())].copy()
print(low_sites_merged_df.shape, low_sites_fb_tw_df.shape)

(340067, 25) (28395, 25)


In [207]:
low_sites_fb_tw_df['title'] = \
low_sites_fb_tw_df['tw_text'].str.replace('\n', ' ') \
.str.replace('http[\:a-zA-Z\/\.0-9]*|\#[a-zA-Z]*|\s{2,}', '', regex=True)
low_sites_fb_tw_df.insert(2, 'article_title', low_sites_fb_tw_df.pop('title'))
print(low_sites_fb_tw_df.isna().sum())
low_sites_fb_tw_df.head()

site                 0
article_url          0
article_title        0
article_text         0
fb_in_id             0
fb_url               0
fb_text              0
fb_datetime          0
fb_likes             0
fb_comments          0
total_fb_posts       0
total_fb_likes       0
total_fb_comments    0
tw_in_id             0
tw_url               0
tw_text              0
tw_datetime          0
tw_retweets          0
tw_mentions          0
total_tweets         0
total_retweets       0
total_mentions       0
site_tw_followers    0
site_tw_following    0
total_followers      0
total_following      0
dtype: int64


,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
187,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%80%cf%...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24218.0,https://www.facebook.com/114954161944042/posts...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,2016-03-05T13:25:11Z,15.0,0.0,...,2016-03-05T13:25:20Z,2.0,68.0,3.0,4.0,68.0,8327.0,347.0,66371.0,2692.0
188,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%bc%ce%...,Μιχαήλ Τσίριμπας ο ακούραστος δάσκαλος ! ! ! -...,Διατηρεί τη μεγαλύτερη Ιστορική και Θρησκευτ...,24219.0,https://www.facebook.com/114954161944042/posts...,Μιχαήλ Τσίριμπας ο ακούραστος δάσκαλος! - Pent...,2016-03-04T08:50:15Z,2.0,0.0,...,2016-03-04T06:50:26Z,0.0,68.0,1.0,0.0,68.0,8328.0,347.0,65431.0,347.0
189,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Τέλος και επίσημα ο Σταυρός από τη σημαία!,"Δημοσίευση: 11 Μαρτίου 2016, 11:24 πμ \n ...",24224.0,https://www.facebook.com/114954161944042/posts...,Τέλος και επίσημα ο Σταυρός από τη σημαία! - P...,2016-03-11T11:46:12Z,10.0,4.0,...,2016-03-11T09:46:24Z,0.0,68.0,2.0,0.0,68.0,8328.0,347.0,66052.0,1789.0
190,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Το κοινό μυστικό των τριών Ευαγγελίων - Γέροντ...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24231.0,https://www.facebook.com/114954161944042/posts...,Το κοινό μυστικό των τριών Ευαγγελίων - Γέροντ...,2016-03-15T14:45:12Z,1.0,0.0,...,2016-03-15T14:45:23Z,0.0,68.0,1.0,0.0,68.0,8329.0,347.0,65432.0,347.0
191,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%b2%ce%...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821,Συμπληρώνονται φέτος 195 χρόνια από την Έναρ...,24245.0,https://www.facebook.com/114954161944042/posts...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821 ...,2016-03-25T00:25:11Z,9.0,1.0,...,2016-03-25T00:25:22Z,1.0,68.0,2.0,2.0,68.0,8328.0,347.0,66232.0,2828.0


In [211]:
low_sites_fb_tw_df.to_csv('0_csvs_for_IQ_feed/low_sites.csv', index=False)

In [212]:
low_sites_df = pd.read_csv('0_csvs_for_IQ_feed/low_sites.csv', lineterminator='\n')
low_sites_df.head()

,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%80%cf%...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24218.0,https://www.facebook.com/114954161944042/posts...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,2016-03-05T13:25:11Z,15.0,0.0,...,2016-03-05T13:25:20Z,2.0,68.0,3.0,4.0,68.0,8327.0,347.0,66371.0,2692.0
1,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%bc%ce%...,Μιχαήλ Τσίριμπας ο ακούραστος δάσκαλος ! ! ! -...,Διατηρεί τη μεγαλύτερη Ιστορική και Θρησκευτ...,24219.0,https://www.facebook.com/114954161944042/posts...,Μιχαήλ Τσίριμπας ο ακούραστος δάσκαλος! - Pent...,2016-03-04T08:50:15Z,2.0,0.0,...,2016-03-04T06:50:26Z,0.0,68.0,1.0,0.0,68.0,8328.0,347.0,65431.0,347.0
2,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Τέλος και επίσημα ο Σταυρός από τη σημαία!,"Δημοσίευση: 11 Μαρτίου 2016, 11:24 πμ \n ...",24224.0,https://www.facebook.com/114954161944042/posts...,Τέλος και επίσημα ο Σταυρός από τη σημαία! - P...,2016-03-11T11:46:12Z,10.0,4.0,...,2016-03-11T09:46:24Z,0.0,68.0,2.0,0.0,68.0,8328.0,347.0,66052.0,1789.0
3,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Το κοινό μυστικό των τριών Ευαγγελίων - Γέροντ...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24231.0,https://www.facebook.com/114954161944042/posts...,Το κοινό μυστικό των τριών Ευαγγελίων - Γέροντ...,2016-03-15T14:45:12Z,1.0,0.0,...,2016-03-15T14:45:23Z,0.0,68.0,1.0,0.0,68.0,8329.0,347.0,65432.0,347.0
4,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%b2%ce%...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821,Συμπληρώνονται φέτος 195 χρόνια από την Έναρ...,24245.0,https://www.facebook.com/114954161944042/posts...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821 ...,2016-03-25T00:25:11Z,9.0,1.0,...,2016-03-25T00:25:22Z,1.0,68.0,2.0,2.0,68.0,8328.0,347.0,66232.0,2828.0


# **Top articles based on engagement**

### NEW

**High quality**

In [221]:
high_top_75_df = \
high_sites_df[(high_sites_df['total_fb_posts'].ge(high_sites_df['total_fb_posts'].quantile(q=.75))) &
                    (high_sites_df['total_fb_likes'].ge(high_sites_df['total_fb_likes'].quantile(q=.75))) &
                    (high_sites_df['total_fb_comments'].ge(high_sites_df['total_fb_comments'].quantile(q=.75))) &
                    (high_sites_df['total_tweets'].ge(high_sites_df['total_tweets'].quantile(q=.75))) & 
                    (high_sites_df['total_retweets'].ge(high_sites_df['total_retweets'].quantile(q=.75)))
                    ]

print(high_top_75_df.site.nunique())
print(high_top_75_df.site.unique())
high_top_75_df.shape

24
['iefimerida' 'newsbeast' 'skai' 'tovima' '24sports' 'zougla' 'star'
 'parapolitika' 'antinews' 'enikos' 'freepen' 'news247' 'balla' 'infokids'
 'cyclades24' 'dimokratiki' 'protothema' 'newspao' 'left' 'gazzetta'
 'aek365' 'real' 'bizznews' 'newsit']


(5011, 26)

In [216]:
high_top_75_df.to_csv('0_csvs_for_IQ_feed/high_sites_top_75.csv', index=False)
high_top_75_df = pd.read_csv('0_csvs_for_IQ_feed/high_sites_top_75.csv', lineterminator='\n')
high_top_75_df.head()

,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/257627/kathizisi...,Καθίζηση στον ιδιωτικό τομέα – 1 εκατ. εργαζόμ...,Συνθήκες «ζούγκλας» επικρατούν στην αγορά εργ...,23757.0,https://www.facebook.com/106952522717824/posts...,Καθίζηση στον ιδιωτικό τομέα – 1 εκατ. εργαζόμ...,2016-03-21T08:04:25Z,23.0,13.0,...,2016-03-21T07:58:17Z,4.0,25854.0,7.0,46.0,25874.0,170136.0,4250.0,297893.0,15502.0
1,iefimerida,http://www.iefimerida.gr/news/258683/koyroympl...,Κουρουμπλής: Αντίχριστος όποιος παπάς δεν θέλε...,Κατά του παπά που απείλησε τους πρόσφυγες στη...,24726.0,https://www.facebook.com/106952522717824/posts...,Κουρουμπλής: Αντίχριστος όποιος παπάς δεν θέλε...,2016-03-26T10:15:11Z,8.0,41.0,...,2016-03-26T10:23:17Z,4.0,25854.0,4.0,12.0,25854.0,170136.0,4250.0,276712.0,7648.0
2,iefimerida,http://www.iefimerida.gr/news/258677/ragise-i-...,«Ράγισε» η Ολλανδία στο 14ο λεπτό - Ετσι τίμησ...,Το φιλικό παιχνίδι ανάμεσα στην Ολλανδία και ...,24727.0,https://www.facebook.com/106952522717824/posts...,«Ράγισε» η Ολλανδία στο 14ο λεπτό - Ετσι τίμησ...,2016-03-26T09:35:31Z,9.0,1.0,...,2016-03-26T09:35:49Z,2.0,25854.0,3.0,6.0,25854.0,170140.0,4250.0,264082.0,9985.0
3,iefimerida,http://www.iefimerida.gr/news/265615/vrohi-tro...,"Βροχή τροπολογιών στη Βουλή -Προσλήψεις, ρύθμι...",Εικοσιέξι τροπολογίες έφεραν οι υπουργοί και ...,28472.0,https://www.facebook.com/106952522717824/posts...,"Βροχή τροπολογιών στη Βουλή -Προσλήψεις, ρύθμι...",2016-05-06T14:55:03Z,10.0,4.0,...,2016-05-06T14:49:02Z,0.0,25854.0,5.0,0.0,25921.0,170140.0,4250.0,266421.0,10801.0
4,iefimerida,http://www.iefimerida.gr/news/274037/pire-kefa...,Πήρε κεφάλι το Brexit – Μπροστά με 53% σε βραδ...,Οι κάλπες έκλεισαν στη Βρετανία και όλοι περι...,34531.0,https://www.facebook.com/106952522717824/posts...,Πήρε κεφάλι το Brexit Μπροστά με 53% σε βραδι...,2016-06-24T04:06:45Z,11.0,3.0,...,2016-06-24T04:07:15Z,0.0,25854.0,4.0,0.0,51712.0,170140.0,4250.0,476976.0,15073.0


**Low quality**

In [245]:
low_top_66_df = \
low_sites_df[(low_sites_df['total_fb_posts'].ge(low_sites_df['total_fb_posts'].quantile(q=.66))) &
                    (low_sites_df['total_fb_likes'].ge(low_sites_df['total_fb_likes'].quantile(q=.66))) &
                    (low_sites_df['total_fb_comments'].ge(low_sites_df['total_fb_comments'].quantile(q=.66))) &
                    (low_sites_df['total_tweets'].ge(low_sites_df['total_tweets'].quantile(q=.66))) & 
                    (low_sites_df['total_retweets'].ge(low_sites_df['total_retweets'].quantile(q=.66)))
                    ]

print(low_top_66_df.site.nunique())
print(low_top_66_df.site.unique())
low_top_66_df.shape

14
['pentapostagma' 'politis' 'newsbomb' 'cretalive' 'sportdog' 'contra'
 'dialogos' 'nooz' 'babyradio' 'matrix24' 'fthis' '24wro' 'piraeuspress'
 'agriniopress']


(4517, 26)

In [242]:
low_top_66_df.to_csv('0_csvs_for_IQ_feed/low_sites_top_66.csv', index=False)
low_top_66_df = pd.read_csv('0_csvs_for_IQ_feed/low_sites_top_66.csv', lineterminator='\n')
low_top_66_df.head()

,site,article_url,article_title,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%80%cf%...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,Ομιλία Γέροντος Νεκταρίου Μουλατσιώτη με θέμ...,24218.0,https://www.facebook.com/114954161944042/posts...,Πως θα φέρω την οικογένειά μου κοντά στην Εκκλ...,2016-03-05T13:25:11Z,15.0,0.0,...,2016-03-05T13:25:20Z,2.0,68.0,3.0,4.0,68.0,8327.0,347.0,66371.0,2692.0
1,pentapostagma,http://www.pentapostagma.gr/2016/03/%cf%84%ce%...,Τέλος και επίσημα ο Σταυρός από τη σημαία!,"Δημοσίευση: 11 Μαρτίου 2016, 11:24 πμ \n ...",24224.0,https://www.facebook.com/114954161944042/posts...,Τέλος και επίσημα ο Σταυρός από τη σημαία! - P...,2016-03-11T11:46:12Z,10.0,4.0,...,2016-03-11T09:46:24Z,0.0,68.0,2.0,0.0,68.0,8328.0,347.0,66052.0,1789.0
2,pentapostagma,http://www.pentapostagma.gr/2016/03/%ce%b2%ce%...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821,Συμπληρώνονται φέτος 195 χρόνια από την Έναρ...,24245.0,https://www.facebook.com/114954161944042/posts...,ΒΙΝΤΕΟ: Ο π. Γ. Μεταλληνός μιλάει για το 1821 ...,2016-03-25T00:25:11Z,9.0,1.0,...,2016-03-25T00:25:22Z,1.0,68.0,2.0,2.0,68.0,8328.0,347.0,66232.0,2828.0
3,pentapostagma,http://www.pentapostagma.gr/2016/04/%cf%80%cf%...,"Πως ""κατάντησε "" το τουρκικό άρμα μάχης Μ-60 S...",Μαχητές του «Ισλαμικού Κράτους» κατάφεραν να...,24307.0,https://www.facebook.com/114954161944042/posts...,"Πως ""κατάντησε "" το τουρκικό άρμα μάχης Μ-60 S...",2016-04-20T17:21:02Z,16.0,0.0,...,2016-04-20T14:21:12Z,0.0,68.0,2.0,0.0,136.0,8326.0,346.0,130851.0,692.0
4,pentapostagma,http://www.pentapostagma.gr/2016/06/%ce%ad%ce%...,"Έβαλε γύρω από τα μάτια της Κουρκουμά και, 5 η...",Ο κουρκουμάς είναι ένα απίστευτα υγιές μπαχα...,24386.0,https://www.facebook.com/114954161944042/posts...,"Έβαλε γύρω από τα μάτια της Κουρκουμά και, 5 η...",2016-06-05T21:03:20Z,9.0,0.0,...,2016-06-05T18:03:27Z,0.0,68.0,2.0,0.0,68.0,8328.0,347.0,87696.0,617.0


**code for removing \n etc**

In [ ]:
#high_df['article_text'] = \
#high_df['article_text'].str.replace('\xa0', ' ').str.replace('\r', ' ')\
#.str.replace('\n', ' ').str.replace('\t', ' ').str.replace('    ', ' ')\
#.str.replace('   ', ' ').str.replace('  ', ' ')

### OLD

In [174]:
high_sites_merged_df.columns

Index(['site', 'article_url', 'article_text', 'fb_in_id', 'fb_url', 'fb_text',
       'fb_datetime', 'fb_likes', 'fb_comments', 'total_fb_posts',
       'total_fb_likes', 'total_fb_comments', 'tw_in_id', 'tw_url', 'tw_text',
       'tw_datetime', 'tw_retweets', 'tw_mentions', 'total_tweets',
       'total_retweets', 'total_mentions', 'site_tw_followers',
       'site_tw_following', 'total_followers', 'total_following'],
      dtype='object')

**High quality sites**

In [175]:
high_sites_merged_df.describe()

,fb_in_id,fb_likes,fb_comments,total_fb_posts,total_fb_likes,total_fb_comments,tw_in_id,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
count,2.125460e+05,212546.000000,212546.000000,212546.000000,212546.00000,212546.000000,4.204990e+05,420499.000000,420499.000000,420499.000000,420499.000000,4.204990e+05,420499.000000,420499.000000,4.204990e+05,4.204990e+05
mean,1.570239e+06,36.239412,2.728007,1.639739,41.12290,3.035352,1.384909e+06,0.264441,2378.139261,2.162964,21.506508,1.500286e+04,125853.166333,1733.894492,2.755803e+05,7.136337e+03
std,8.872417e+05,314.684070,11.466680,2.175053,337.35093,12.772471,9.318916e+05,2.977693,6693.512533,4.660189,1176.190497,9.672142e+04,152628.580312,2221.703251,1.165069e+06,2.991931e+04
min,2.011200e+04,0.000000,0.000000,1.000000,0.00000,0.000000,1.658700e+04,0.000000,0.000000,1.000000,0.000000,0.000000e+00,10.000000,2.000000,2.390000e+02,2.000000e+00
25%,6.658142e+05,0.000000,0.000000,1.000000,0.00000,0.000000,5.992735e+05,0.000000,0.000000,1.000000,0.000000,0.000000e+00,6434.000000,305.000000,1.526800e+04,7.720000e+02
50%,1.407126e+06,2.000000,0.000000,1.000000,2.00000,0.000000,1.220886e+06,0.000000,22.000000,1.000000,0.000000,1.340000e+02,66950.000000,860.000000,9.898500e+04,1.957000e+03
75%,2.625338e+06,9.000000,1.000000,1.000000,11.00000,1.000000,2.181840e+06,0.000000,647.000000,2.000000,0.000000,1.697000e+03,169933.000000,2243.000000,2.887160e+05,4.560000e+03
max,2.978184e+06,48310.000000,778.000000,109.000000,48506.00000,886.000000,3.016191e+06,1497.000000,27313.000000,367.000000,313455.000000,9.098318e+06,703322.000000,9129.000000,9.838715e+07,2.702846e+06


**All metrics >= 75%**

In [176]:
high_top_engagement_df = \
high_sites_merged_df[(high_sites_merged_df['total_fb_posts'].ge(high_sites_merged_df['total_fb_posts'].quantile(q=.75))) &
                    (high_sites_merged_df['total_fb_likes'].ge(high_sites_merged_df['total_fb_likes'].quantile(q=.75))) &
                    (high_sites_merged_df['total_fb_comments'].ge(high_sites_merged_df['total_fb_comments'].quantile(q=.75))) &
                    (high_sites_merged_df['total_tweets'].ge(high_sites_merged_df['total_tweets'].quantile(q=.75))) & 
                    (high_sites_merged_df['total_retweets'].ge(high_sites_merged_df['total_retweets'].quantile(q=.75)))
                    ]


In [177]:
high_top_engagement_df.site.nunique()

24

In [178]:
high_top_engagement_df.site.unique()

array(['iefimerida', 'newsbeast', 'skai', 'tovima', '24sports', 'zougla',
       'star', 'parapolitika', 'antinews', 'enikos', 'freepen', 'news247',
       'balla', 'infokids', 'cyclades24', 'dimokratiki', 'protothema',
       'newspao', 'left', 'gazzetta', 'aek365', 'real', 'bizznews',
       'newsit'], dtype=object)

**Low quality sites**

In [179]:
low_sites_merged_df.describe()

,fb_in_id,fb_likes,fb_comments,total_fb_posts,total_fb_likes,total_fb_comments,tw_in_id,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
count,1.758780e+05,175878.000000,175878.000000,175878.000000,175878.000000,175878.000000,1.925840e+05,192584.000000,192584.000000,192584.000000,192584.000000,192584.000000,192584.000000,192584.000000,1.925840e+05,192584.000000
mean,9.971415e+05,23.803034,1.622414,1.769789,28.226600,1.882339,1.029858e+06,0.248759,567.437487,1.795331,5.166452,971.224728,38516.506106,1857.425368,1.377584e+05,3815.776581
std,5.602888e+05,158.246266,8.063534,2.436762,170.933451,8.937353,6.012518e+05,1.066635,1379.447668,2.580995,419.280876,3072.739361,50587.872747,2702.318284,2.332495e+05,9245.501202
min,2.392000e+04,0.000000,0.000000,1.000000,0.000000,0.000000,1.920000e+02,0.000000,0.000000,1.000000,0.000000,0.000000,143.000000,5.000000,1.430000e+02,5.000000
25%,5.411148e+05,0.000000,0.000000,1.000000,0.000000,0.000000,5.268815e+05,0.000000,0.000000,1.000000,0.000000,0.000000,5456.000000,419.000000,1.110200e+04,473.000000
50%,1.066304e+06,2.000000,0.000000,1.000000,2.000000,0.000000,1.153890e+06,0.000000,0.000000,1.000000,0.000000,0.000000,11747.000000,715.000000,2.354300e+04,821.000000
75%,1.384340e+06,10.000000,0.000000,2.000000,12.000000,1.000000,1.460252e+06,0.000000,570.000000,2.000000,0.000000,1112.000000,71067.000000,2125.000000,1.508810e+05,3871.000000
max,2.032515e+06,21097.000000,576.000000,78.000000,21114.000000,633.000000,2.050823e+06,90.000000,10035.000000,367.000000,103350.000000,437050.000000,184003.000000,10898.000000,5.228302e+06,894705.000000


**All metrics >= 67%**

In [180]:
low_top_engagement_df = \
low_sites_merged_df[(low_sites_merged_df['total_fb_posts'].ge(low_sites_merged_df['total_fb_posts'].quantile(q=.67))) &
                    (low_sites_merged_df['total_fb_likes'].ge(low_sites_merged_df['total_fb_likes'].quantile(q=.67))) &
                    (low_sites_merged_df['total_fb_comments'].ge(low_sites_merged_df['total_fb_comments'].quantile(q=.67))) &
                    (low_sites_merged_df['total_tweets'].ge(low_sites_merged_df['total_tweets'].quantile(q=.67))) & 
                    (low_sites_merged_df['total_retweets'].ge(low_sites_merged_df['total_retweets'].quantile(q=.67)))
                     ]
                     

In [181]:
low_top_engagement_df.site.nunique()

16

In [182]:
low_top_engagement_df.site.unique()

array(['pentapostagma', 'politis', 'newsbomb', 'cretalive', 'sportdog',
       'contra', 'dialogos', 'nooz', 'babyradio', 'matrix24', 'fthis',
       'triklopodia', '24wro', 'piraeuspress', 'sdna', 'agriniopress'],
      dtype=object)

**Notes:** 

Engagement factors: total fb posts, total tweets, total fb likes, total fb comments, total retweets

High quality
- βρέθηκαν 6258 άρθρα από 24 sites (high quality, όλα τα metrics του engagement >= 75%)
- όνομα df: high_top_engagement_df

Low quality:
- βρέθηκαν 8410 άρθρα από 16 sites (low quality, όλα τα metrics του engagement >= 67%)
- όνομα df: low_top_engagement_df

In [183]:
high_top_engagement_df.to_csv('top_high_engagement_sites.csv', index=False)

In [184]:
high_top_engagement_df.columns

Index(['site', 'article_url', 'article_text', 'fb_in_id', 'fb_url', 'fb_text',
       'fb_datetime', 'fb_likes', 'fb_comments', 'total_fb_posts',
       'total_fb_likes', 'total_fb_comments', 'tw_in_id', 'tw_url', 'tw_text',
       'tw_datetime', 'tw_retweets', 'tw_mentions', 'total_tweets',
       'total_retweets', 'total_mentions', 'site_tw_followers',
       'site_tw_following', 'total_followers', 'total_following'],
      dtype='object')

In [187]:
high_top_engagement_df = pd.read_csv('top_high_engagement_sites.csv', lineterminator='\n')

In [188]:
high_top_engagement_df

,site,article_url,article_text,fb_in_id,fb_url,fb_text,fb_datetime,fb_likes,fb_comments,total_fb_posts,...,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/257627/kathizisi...,Συνθήκες «ζούγκλας» επικρατούν στην αγορά εργ...,23757.0,https://www.facebook.com/106952522717824/posts...,Καθίζηση στον ιδιωτικό τομέα – 1 εκατ. εργαζόμ...,2016-03-21T08:04:25Z,23.0,13.0,2.0,...,2016-03-21T07:58:17Z,4.0,25854.0,7.0,46.0,25874.0,170136.0,4250.0,297893.0,15502.0
1,iefimerida,http://www.iefimerida.gr/news/274037/pire-kefa...,Οι κάλπες έκλεισαν στη Βρετανία και όλοι περι...,34531.0,https://www.facebook.com/106952522717824/posts...,Πήρε κεφάλι το Brexit Μπροστά με 53% σε βραδι...,2016-06-24T04:06:45Z,11.0,3.0,1.0,...,2016-06-24T04:07:15Z,0.0,25854.0,4.0,0.0,51712.0,170140.0,4250.0,476976.0,15073.0
2,iefimerida,http://www.iefimerida.gr/news/274823/brexit-ka...,Η Αγγλία δεν ζει και τις καλύτερες ημέρες της...,35803.0,https://www.facebook.com/106952522717824/posts...,Brexit (και) στο EURO από την απίστευτη Ισλανδ...,2016-06-27T23:50:48Z,45.0,2.0,1.0,...,2016-06-27T23:50:52Z,0.0,25854.0,4.0,0.0,51753.0,170182.0,4250.0,447081.0,9094.0
3,iefimerida,http://www.iefimerida.gr/news/275078/i-foyreir...,Τα Mad Video Awards είναι η μοναδική ευκαιρία...,36709.0,https://www.facebook.com/106952522717824/posts...,Η Φουρέιρα αντέγραψε το «γυμνό» φόρεμα της Μπέ...,2016-06-29T23:55:00Z,12.0,15.0,1.0,...,2016-06-29T09:17:57Z,0.0,25854.0,2.0,0.0,51708.0,170140.0,4250.0,445627.0,8500.0
4,iefimerida,http://www.iefimerida.gr/news/275674/egine-rez...,Έγινε διάσημη για ένα κυρίως πράγμα... για τι...,37702.0,https://www.facebook.com/106952522717824/posts...,Εγινε ρεζίλι: Η απομυθοποίηση της Καρντάσιαν- ...,2016-07-02T01:01:00Z,9.0,7.0,2.0,...,2016-07-01T16:40:10Z,0.0,25854.0,5.0,0.0,51708.0,169976.0,4253.0,553159.0,10007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6253,newsit,http://www.newsit.gr/kosmos/O-Erntogan-apagore...,"- Νέος ""πόλεμος"" ανάμεσα στην Τουρκία και τη ...",2974460.0,https://www.facebook.com/170069547711/posts/10...,Ο Ερντογάν απαγόρευσε τα Χριστούγεννα στους Γε...,2016-12-18T19:03:33Z,13.0,11.0,3.0,...,2016-12-18T19:03:51Z,0.0,576.0,5.0,0.0,576.0,121252.0,391.0,313890.0,10386.0
6254,newsit,http://www.newsit.gr/lifestyle/Otan-o-George-M...,Tα βίντεο που θα σας συγκινήσουν! \r\n\r\n...,2976759.0,https://www.facebook.com/170069547711/posts/10...,Όταν ο George Michael τραγούδησε στην Αθήνα με...,2016-12-26T10:13:33Z,148.0,5.0,4.0,...,2016-12-26T10:14:02Z,0.0,576.0,6.0,0.0,658.0,121252.0,391.0,596732.0,3034.0
6255,newsit,http://www.newsit.gr/oikonomia/Stin-dimosiotit...,- Στην επιστολή η κυβέρνηση δεσμεύεται αν βγο...,2977069.0,https://www.facebook.com/170069547711/posts/10...,Στην δημοσιότητα η επιστολή Τσακαλώτου! ΕΚΤΑΚΤ...,2016-12-27T14:56:27Z,10.0,9.0,2.0,...,2016-12-27T14:54:32Z,0.0,576.0,2.0,0.0,576.0,121252.0,391.0,338629.0,766.0
6256,newsit,http://www.newsit.gr/kairos/Kairos-Ektakto-del...,- Πυκνές χιονοπτώσεις σε όλη της χώρα από την...,2977072.0,https://www.facebook.com/170069547711/posts/10...,Καιρός: Εκτακτο δελτίο επιδείνωσης - Θα χιονίσ...,2016-12-27T13:44:35Z,11.0,2.0,9.0,...,2016-12-27T13:45:12Z,0.0,576.0,2.0,0.0,576.0,121288.0,391.0,792216.0,1294.0
